# 从 DFT 到 MD｜超详细「深度势能」材料计算上手指南

<a href="https://nb.bohrium.dp.tech/detail/6116401077" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>


<div class="alert alert-info">
    <details>
<summary>©️Copyright @ Authors</summary>
<br>
    <i>日期：2023-05-09</i><br/>
    <i>共享协议：本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i>

</details>
</div>

<span style="color:purple">**这是目前最完整地覆盖从第一性原理计算到分子动力学模拟的超详细「深度势能」材料计算上手指南，主要面向需要入门深度势能材料计算的零基础用户。**</span>

或者你仅需要 10 分钟快速了解 DeePMD-kit，可戳 👉 [DeePMD-kit｜快速开始](https://nb.bohrium.dp.tech/detail/3313403083)

**在 [Bohrium Notebook](https://bohrium-doc.dp.tech/docs/userguide/Notebook) 界面，你可以点击界面上方蓝色按钮 `开始连接`，选择 `abacus3.2.1-deepmd` 镜像及 `c32_m128_CPU` 节点配置，稍等片刻即可运行。**

<div style="padding: 10px; background-color: #f5f5f5; color: black; margin-bottom: 15px; border-radius: 30px;">
    <p style='padding: 10px'>本教程可一键运行，<b>完整运行该教程需要约 20 小时</b> （ c32_m128_CPU 节点配置下）。如您希望快速运行，可以<b>跳过注释了 Time Warning 的单元，这将大幅缩短运行时间至几分钟。</b></p>
        <p style='padding-left: 10px; padding-right: 10px;'>请放心，这并不会影响你的学习体验。我们已为您运行并准备好了部分运行结果文件。</p>
</div>

<div style="width:auto; height:2px; background:linear-gradient(244deg,rgba(0,0,0,0) 0%,rgba(0,0,0,0.5) 50%,rgba(0,0,0,1) 100%)"></div>

## 背景

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S1005030220309075-fx1_lrg.jpg" alt="LiCl" width="600">

在本教程中，我们将以 LiCl 熔体为例，详细介绍深度势能（Deep Potential）模型的训练和应用。深度势能模型是一种基于机器学习的方法，用于精确描述原子间相互作用的势能面。通过训练这种模型，我们可以高效地模拟材料的结构、动力学和热力学性质。具体包括以下示例：

* 章节 1: LAMMPS 经典分子动力学模拟
* 章节 2: ABACUS 第一性原理计算
* 章节 3: DeePMD-kit 深度势能模型训练
* 章节 4: DP-GEN 构建训练数据并生成深度势模型
* 章节 5: LAMMPS 深度势能分子动力学研究

总的来说，通过结合经典分子动力学模拟（如 LAMMPS）、第一性原理计算（如 ABACUS）和深度势能模型构建（DeePMD-kit 和 DP-GEN），我们可以多角度地研究 LiCl 熔体的性质。这些方法和工具的组合为研究复杂材料系统提供了强大的支持，帮助我们更好地理解材料在不同条件下的性能。

### 开始之前

我们很热切地希望与你分享我们的知识，但是现在：

**你不需要了解所有事（至少目前是）。** 你的目标是从头到尾完成本教程并获得结果。你不需要在第一次尝试时就了解所有内容。边学习边写下你的问题。使用丰富的 API 文档来了解你正在使用的所有功能。

**你不需要知道算法是如何工作的。** 了解各种计算模拟的优点和局限性以及如何配置材料计算参数非常重要。但是学习可以稍后进行。你需要在很长一段时间内慢慢建立这种计算模拟知识，并且你可以阅读大量教程来复习材料计算项目的步骤。今天，首先要熟悉这个平台。

了解以下基础概念可以帮助你更快速地理解本文：

* 分子动力学 (Molecular Dynamics, MD)，如果你尚未了解，推荐阅读：
    * [快速了解分子动力学](https://bohrium.dp.tech/notebook/e46f760ebd5f49dba710f5000c1c5eb0)
* 密度泛函理论 (Density Fuctional Theory，DFT)，如果你尚未了解，推荐阅读：
    * [快速了解密度泛函理论](https://nb.bohrium.dp.tech/detail/7317630945)

## 章节 1：LAMMPS 经典分子动力学模拟

对于 LiCl 熔体，可以使用 LAMMPS（Large-scale Atomic/Molecular Massively Parallel Simulator）软件进行经典分子动力学模拟。[LAMMPS](https://www.lammps.org) 是一个高度灵活且可扩展的分子动力学模拟软件，支持多种原子间势和模拟条件。在这个示例中，我们使用经典 BMH 势函数模拟 LiCl 熔体的结构和动力学性质。

本章节采用 DeePMD-kit(2.2.1) 软件包中预置的 LAMMPS 程序完成。

### 本章节目标

在学习本章节后，你将能够：

- 了解 LAMMPS 的输入和输出文件；
- 能够为 LiCl 熔体撰写 LAMMPS MD 模拟输入文件；
- 计算 LiCl 熔体的微观结构和扩散性质。

### 1.1 下载教程资源

在本章节中，我们以 LiCl 熔体分子为例，进行 LAMMPS 经典分子动力学模拟。我们已经在 *LiCl_DP_Tutorial_Example/chapter1* 中准备了需要的文件，你可以点击左侧数据集查看：

In [1]:
# 出于安全考虑，我们没有数据集所在文件夹的写入权限，因此我们将其复制到 `/data/` 目录下:
! cp -nr /bohr/ /personal/

# 我们在这里定义一些路径，并切换到工作路径，方便后续调用：
import os
bohr_dataset_url = "/bohr/licl-dp-tutorial-unuc/v1/"  # 数据集的 url 可从左侧数据集复制
work_path = os.path.join("/personal", bohr_dataset_url[1:])  # 进行一个切片以删除上述路径中最开始的“/”
os.chdir(work_path)
print(f"当前路径为：{os.getcwd()}")

当前路径为：/data/bohr/licl-dp-tutorial-unuc/v1


在 *LiCl_DP_Tutorial_Example/chapter1* 文件夹包含以下文件：

- `licl.in`: LAMMPS 输入文件，用于控制 LAMMPS MD 模拟的细节；
- `licl.data`: 用于存放 MD 模拟的初始构型；

### 1.2 LAMMPS 输入文件

输入文件的作用是告诉 LAMMPS 软件其应该如何计算。通常将文件命名为“input.in”，其中“input”可以是任何描述性名称，以便于识别该模拟的类型或参数。

下面是一个 LiCl 熔体 LAMMPS 分子动力学模拟输入文件的示例：

```
# this input script is for simulating a 3d LiCl melt at 900K using LAMMPS.

# initialize simulation settings
units           metal
boundary        p p p
atom_style      charge

# define the simulation cell
read_data       licl.data
group           Li  type 1
group           Cl  type 2
set             type 1 charge 1
set             type 2 charge -1

# set force field
pair_style      born/coul/long 7
pair_coeff      1 1 0.4225000 0.3425 1.632 0.045625 0.01875
pair_coeff      1 2 0.2904688 0.3425 2.401 1.250000 1.50000
pair_coeff      2 2 0.1584375 0.3425 3.170 69.37500 139.375
kspace_style    ewald 1.0e-6 	

# nvt simulation  
velocity        all create 900 23456789
fix             1 all nvt temp 900 900 0.5
timestep        0.001

# rdf calculation 
compute         rdf all rdf 100 1 1 1 2 2 2
fix             2 all ave/time 100 1 100 c_rdf[*] file licl.rdf mode vector

# msd calculation
compute         msd1 Li msd
compute         msd2 Cl msd
fix             3 all ave/time 100 1 100 c_msd1[4] c_msd2[4] file licl.msd

# output
thermo_style    custom step temp pe ke etotal press lx ly lz vol
thermo          1000
dump            1 all custom 1000 licl.dump id type x y z 

log             ./LiCl_DP_Tutorial_Example/Chapter1/Outputs/log.lammps
run             500000
```

让我们来看一下各输入文件中各参数的含义：

- `units metal`：用于设置模拟中使用的单位系统。对于 metal，时间的单位是皮秒（ps），长度的单位是埃（Å），质量的单位是原子质量单位（amu），能量的单位是电子伏特（eV），温度的单位是开尔文（K），压力的单位是巴（bar），速度的单位是埃/皮秒（Å/ps）。

- `boundary p p p`：用于设置模拟的边界条件。在本教程情况下，我们在 x, y, z 三个方向上均使用周期性（periodic）边界条件。p 表示周期性，f (fixed) 表示固定边界条件。

- `atom_style charge`：用于设置原子类型和属性。在这个示例中，我们使用带电原子模型，因此使用了 charge 类型。

- `read_data licl.data`：用于读取数据文件 licl.data，同时也是该案例中 MD 模拟所使用的初始构型。

- `group Li type 1` 和 `group Cl type 2`：用于根据原子类型创建两个组，分别包含元素类型为 1（Li）和元素类型为 2（Cl）的所有原子。

- `set type 1 charge 1` 和 `set type 2 charge -1`：用于设置原子类型 1 和 2 的电荷。在这个示例中，元素类型 1（Li）的原子被设置为带有 +1 电荷，元素类型 2（Cl）的原子被设置为带有 -1 电荷。

- `pair_style born/coul/long 7`：该命令用于设置原子之间的相互作用势函数。在这个示例中，我们使用 Born−Mayer−Huggins 势函数，截断距离设置为 7 Å：

$$ U_{i j}(r)=\frac{q_i q_j}{r}+A_{i j} b \exp \left(\frac{\sigma_{i j}-r}{\rho}\right)-\frac{C_{i j}}{r^6}-\frac{D_{i j}}{r^8} $$

其中第一项描述了离子之间的静电相互作用，$q_i$ 是离子电荷（$q_{Li}$= +1，$q_{Cl}$=-1）；第二项描述由于电子云的重叠引起的短程斥力，$A_{i j}$ 是 Pauling 因子（$A_{Li Li}$=2.00，$A_{Li Cl}$=1.375，$A_{Cl Cl}$=0.75），b 是一个常数（$b=0.338 \times 10^{-19} \mathrm{~J}$），$σ_{i j}$ 是晶体离子半径，而 ρ 是硬度参数 ($ρ_{LiCl}$ = 0.3425 Å)；最后两项对应于偶极-偶极和偶极-四极子色散相互作用，其中 $C_{i j}$和$D_{i j}$ 是色散参数。$r$ 是截断距离（cutoff distance），单位为 Å，超过这个距离的原子间作用将被忽略。

|    |$A_{i j}b$(eV)|$σ_{i j}$(Å)|$C_{i j}$(eV)|$D_{i j}$(eV)|
|----|--------------|------------|-------------|-------------| 
| ++ | 0.4225000    | 1.632      |0.045625     |0.01875      |
| +- | 0.2904688    | 2.401      |1.250000     |1.50000      |
| -- | 0.1584375    | 3.170      |69.37500     |139.375      |

- `pair_coeff`：这些命令用于为相互作用势函数设置参数。每个 pair_coeff 命令将相互作用势函数中的原子类型之间的参数设置为指定值。在这个示例中，我们设置了 1 1、1 2 和 2 2 之间的参数。

- `kspace_style ewald 1.0e-6`：该命令用于设置 Ewald 方法计算长程库仑相互作用。在这个示例中，我们使用了Ewald方法，使用了 1.0e-6 的精度。

- `velocity all create 900 23456789`：该命令用于为模拟系统中的所有原子设置随机速度。

- `fix 1 all nvt temp 900 900 0.5`：该命令用于对模拟系统进行 NVT （等温-等体积）模拟。在这个示例中，我们将模拟系统保持在 900K 的恒定温度下，并使用Nose-Hoover 算法进行温度控制。0.5 是温度阻尼系数。

- `timestep 0.001`：该命令用于设置模拟的时间步长。在这个示例中，时间步长为 0.001 皮秒。（1 秒 = 10^3 毫秒 = 10^6 微秒 = 10^9 纳秒 = 10^12 皮秒）

- `compute rdf all rdf 100 1 1 1 2 2 2`：该命令用于计算模拟系统中两种原子之间的径向分布函数（RDF）。100 表示分 100 个统计区间，1 1 表示 Li-Li RDF，1 2表示Li-Cl RDF，2 2 表示 Cl-Cl RDF。

- `fix 2 all ave/time 100 1 100 c_rdf[*] file licl.rdf mode vector`：该命令用于对计算的RDF数据进行时间平均，并将结果输出到文件中。使用 fix 2 对RDF 数据进行时间平均，100 1 100 分别为 Nevery (每100步计算1次 rdf)，Nrepeat(平均最近1次的计算的 rdf，用于输出)和 Nfrequency(每 100 步输出一次 rdf)， 在这个案例中会每 100 步会输出一次 rdf。使用 c_rdf[*] 表示平均所有 RDF 分量，使用 file licl.rdf 表示将结果输出到名为 licl.rdf 的文件中，使用 mode vector 表示输出向量格式的数据。

- `compute msd1 Li msd` 和 `compute msd2 Cl msd`：这些命令用于计算两种原子在模拟过程中的平均平方位移（MSD）。在这个示例中，我们使用 compute msd1 计算 Li 原子的 MSD，使用compute msd2 计算 Cl 原子的 MSD。

- `fix 3 all ave/time 100 1 100 c_msd1[4] c_msd2[4] file licl.msd`：该命令用于对计算的 MSD 数据进行时间平均，并将结果输出到文件中。使用 fix 3 对 MSD 数据进行时间平均，使用 c_msd1[4] 和 c_msd2[4] 表示平均两种原子的MSD, 使用 file licl.msd 表示将结果输出到名为licl.msd的文件中。

- `thermo_style custom step temp pe ke etotal press lx ly lz vol`：该命令用于设置 LAMMPS 在模拟过程中输出的热力学信息。在这个示例中，我们使用`thermo_style custom` 表示使用自定义的输出格式，使用 `step temp pe ke etotal press lx ly lz vol` 表示要输出的信息，包括模拟步数、温度、势能、动能、总能量、压强以及模拟盒子的尺寸和体积。

- `thermo 1000`：该命令用于设置输出热力学信息的频率。在这个示例中，我们将每 1000 步输出一次热力学信息。

- `dump 1 all custom 1000 licl.dump id type x y z`：该命令用于在模拟过程中输出原子的坐标信息。在这个示例中，使用 `dump 1 all` 表示输出模拟系统中的所有原子，使用 `custom` 表示使用自定义的输出格式，使用 `1000` 表示输出频率，使用 `licl.dump` 表示输出文件的名称，使用 `id type x y z` 表示输出的原子信息，包括原子的 ID、类型以及在 x、y、z 方向上的坐标。

- `run 500000` 命令用于运行 LAMMPS 模拟，进行一定步数的时间演化。在这个示例中，该命令将模拟系统进行 500000 步时间演化。在每个时间步长中，LAMMPS 会根据当前的原子位置、速度和势能计算新的位置、速度和势能。通过这个过程，我们可以观察模拟系统的时间演化行为，比如温度、压力和分子运动轨迹等。需要注意的是，run 命令需要根据实际情况进行设置，以保证模拟过程的充分和准确。

你可以通过以下命令查看你刚才下载的 LAMMPS 案例的输入文件，与上述示例比较一下，有什么发现？

In [3]:
! cd ./LiCl_DP_Tutorial_Example/chapter1/ && cat licl.in

# this input script is for simulating a 3d LiCl melt at 900K using LAMMPS.

# initialize simulation settings
units           metal
boundary        p p p
atom_style      charge

# define the simulation cell
read_data	licl.data
group		Li  type 1
group		Cl  type 2
set 		type 1 charge 1
set 		type 2 charge -1

# set force field
pair_style      born/coul/long 7
pair_coeff	1 1 0.4225000 0.3425 1.632 0.045625 0.01875
pair_coeff	1 2 0.2904688 0.3425 2.401 1.250000 1.50000
pair_coeff	2 2 0.1584375 0.3425 3.170 69.37500 139.375
kspace_style	ewald 1.0e-6 	

# nvt  
velocity        all create 900 23456789
fix             1 all nvt temp 900 900 0.5
timestep        0.001

# rdf calculation 
compute 	 rdf all rdf 100 1 1 1 2 2 2
fix 		 2 all ave/time 100 1 100 c_rdf[*] file licl.rdf mode vector

# msd calculation
compute          msd1 Li msd
compute          msd2 Cl msd
fix              3 all ave/time 100 1 100 c_msd1[4] c_msd2[4] file licl.msd

# output
thermo_style    custom step temp pe ke etotal 

为了缩短运行所需的时间，我们适当降低了运行步数。你有新的想法吗？不要犹豫，快速试试你的新想法。

如果你没有偷懒的话，你已经仔细阅读了 2～3 遍 LAMMPS 输入文件，相信你已经对输入文件有了初步的了解。

还有许多疑惑？不用担心，这是正常的。你需要在接下来的时间里慢慢建立知识。[LAMMPS Commands 官方文档](https://docs.lammps.org/Commands.html) 提供了全面、详细的解释，在需要的时候不要忘了它。

### 1.3 LAMMPS 初始构型文件

下面是一个 LiCl 熔体 LAMMPS MD 模拟初始构型的示例：

```
# LAMMPS data file 
108 atoms
2 atom types
0.0 13.4422702789 xlo xhi
0.0 13.4422702789 ylo yhi
0.0 13.4422702789 zlo zhi

Masses

1 6.941  # Li
2 35.453  # Cl

Atoms  # charge

1 1 0.0 9.10297966 1.4528499842 12.3941898346  
2 1 0.0 11.53647995 2.3037500381 1.6365799904
3 1 0.0 1.3658800125 9.3088798523 4.9590802193
...
106 2 0.0 5.8468399048 2.629529953 3.9059700966
107 2 0.0 7.0047798157 5.3034000397 10.0816297531
108 2 0.0 4.4860801697 11.4718704224 13.3586997986
```

该文件描述模拟系统（LiCl）的基本信息和初始状态。

首先是模拟系统的基本信息：模拟系统中有 108 个原子，两种类型的原子（Li 和 Cl），模拟系统在 x、y、z 三个方向上的盒子大小分别是 0.0 到 13.4422702789，两种类型的原子的质量分别是 6.941 和 35.453。

后面一部分是原子的位置和电荷信息：每行依次表示原子的 ID、类型、电荷以及在 x、y、z 三个方向上的坐标。注意，此处电荷都是 0.0，我们可以在 *licl.in* 中重新规定电荷。

同样地，你可以通过以下命令查看你刚才下载的 LAMMPS 案例的构型数据文件。<span style="color:purple; font-weight:bold">自己动手试一下。</span>

```bash
cat ./LiCl_DP_Tutorial_Example/chapter1/licl.data
```

### 1.4 运行 LAMMPS 经典分子动力学模拟

经典分子动力学（Classical Molecular Dynamics，CMD）使用牛顿力学描述原子或分子间的相互作用。

在了解 licl.in 和 licl.data 文件后，我们可以执行以下命令以启动 LiCl 熔体的 LAMMPS 分子动力学模拟：

In [1]:
# 以下命令判断该环境中是否存在所需模块，如果没有，则使用 pip 快速安装。
! ! if ! pip show deepmd-kit > /dev/null; then pip install deepmd-kit[gpu,cu11,lmp]; fi;

In [2]:
# ############### Time Warning: 3 mins 35 secs ################### 
! cd ./LiCl_DP_Tutorial_Example/chapter1/ && lmp -i licl.in

This LAMMPS executable is in a conda environment, but the environment has
not been activated. Libraries may fail to load. To activate this environment
please see https://conda.io/activation.
LAMMPS (23 Jun 2022 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from /opt/deepmd-kit-2.2.1/lib/deepmd_lmp
Reading data file ...
  orthogonal box = (0 0 0) to (13.44227 13.44227 13.44227)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  108 atoms
  read_data CPU = 0.033 seconds
54 atoms in group Li
54 atoms in group Cl
Setting atom values ...
  54 settings made for charge
Setting atom values ...
  54 settings made for charge
Ewald initialization ...
  using 12-bit tables for long-range coulomb (src/kspace.cpp:342)
  G vector (1/distance) = 0.4944418
  estimated absolute RMS force accuracy = 1.4603535e-05
  estimated relative force accuracy = 1.0141594e-06
  KSpace vectors: actual max1d ma

### 1.5 LAMMPS 分子动力学模拟结果与分析

运行完成后，我们可以看到在当前文件夹下生成了以下几个文件：

- log.lammps：日志文件，记录了模拟过程中的各种输出信息，包括初始系统状态、MD过程中的能量、温度、压力等物理量，以及各种计算的结果。
- licl.dump：轨迹文件，记录了模拟系统在每个时间步长中所有原子的位置、速度等信息。
- licl.rdf：径向分布函数（RDF）文件，记录了每 Nfrequency 步数输出的 RDF。
- licl.msd：均方位移（MSD）文件，记录了模拟系统中离子的均方位移随时间的变化。


对于 licl.dump 文件，我们可以使用 OVITO 等软件对完整轨迹进行可视化。在这里，我们使用 ASE 仅查看最后一帧的图像。了解 ASE 的使用，可以阅读[《快速开始 ASE》](https://nb.bohrium.dp.tech/detail/8841808328)

In [1]:
# 以下命令判断该环境中是否存在所需模块，如果没有，则使用 pip 快速安装。
! if ! /opt/mamba/bin/pip show ase > /dev/null; then /opt/mamba/bin/pip install --upgrade ase; fi;

In [1]:
from ase.io import lammpsrun
from ase.visualize import view
from ase import Atoms

# 导入轨迹文件
traj = lammpsrun.read_lammps_dump('./LiCl_DP_Tutorial_Example/chapter1/licl.dump')

# 定义元素名称列表
elements = ['Li', 'Cl']

# 将元素名称分配给轨迹的原子
# 根据原子类型设置原子名称
for atom in traj:
    atom_type = atom.number - 1  # 原子类型从1开始，而列表索引从0开始
    atom.symbol = elements[atom_type]

print(traj)

view(traj, viewer='x3d') 

Atoms(symbols='Cl54Li54', pbc=True, cell=[13.4422702789, 13.4422702789, 13.4422702789])


<details>
<summary>
<b>了解一下什么是径向分布函数？为什么分子动力学中要计算径向分布函数？</b>
</summary>
<hr/>
径向分布函数（Radial Distribution Function，RDF）是一种描述在固体、液体和气体等多原子系统中，原子或分子间相对位置和距离分布的统计量。它可以用来表示从一个选定的原子或分子出发，与其距离为 r 的其他原子或分子的概率密度。径向分布函数 g(r) 可以表示为：

$$
g(r) = \frac{N}{V} \times \frac{1}{\rho} \times \frac{dN}{dr}
$$

其中，N 是距离 r 范围内的原子或分子数，V 是体积，ρ 是原子或分子的平均密度，$(\frac{dN}{dr})$ 是在距离 r 附近的原子或分子数增加率。

我们以氧气为例。在氧气中，氧分子之间主要通过范德华力相互作用。我们可以计算氧分子间的径向分布函数来分析这种相互作用。

考虑氧气中的一个氧分子，以其为中心，计算距离 r 内其他氧分子的概率密度。径向分布函数 g(r) 描述了在距离 r 处找到另一个氧分子的概率密度，相对于在完全随机分布的情况下找到另一个氧分子的概率密度。

在氧气中计算氧分子之间的径向分布函数 g(r)，我们可以观察到：

1. 当 r 很小时，g(r) 接近 0，这意味着氧分子不可能位于非常接近的位置，这是由于它们之间的排斥力。

2. 随着 r 的增加，g(r) 的值会出现一个峰值，这个峰值对应于氧分子间最可能的距离，即范德华力最强的距离。

3. 当 r 继续增加时，g(r) 将逐渐趋于 1，这意味着在较大的距离范围内，氧分子的分布接近随机分布。

当 r 增加时，g(r) 趋近于1的原因是，g(r) 是相对于完全随机分布的概率密度。在距离很远时，原子或分子之间的相互作用变得微弱，分子分布趋近于随机。

当我们提到 g(r) 趋近于1时，我们是在描述这样一个现象：在较大的距离范围内，分子之间的相互作用变得不那么重要，因此它们的分布接近于随机分布。在这种情况下，实际系统中找到一个分子的概率与随机分布中找到一个分子的概率相近，因此 g(r) 接近1。

需要注意的是，g(r) 本身并不是概率，而是概率密度。在径向分布函数 g(r) 的情况下，它表示的是在距离 r 附近找到另一个分子的相对可能性，因此是一个概率密度函数。g(r) 的值并不直接反映在某个距离 r 处找到分子的概率，而是相对于随机分布的概率密度。因此，当 r 增加时，g(r) 不会无限增大，而是趋于一个稳定值，即 1。这表明分子分布在远离中心分子的区域趋近于随机。

从这个简单的例子中，我们可以看到径向分布函数是如何帮助我们了解氧分子间相互作用和分布的。同样的方法也可以应用于其他气体系统，以揭示其原子或分子间的相互作用和结构性质。

在分子动力学（Molecular Dynamics，MD）模拟中，径向分布函数是一种重要的工具，因为它可以提供关于系统的结构信息，帮助我们了解原子或分子间的相互作用和系统的组织方式。计算径向分布函数可以帮助我们：

1. 描述和量化原子或分子之间的距离分布，进而分析分子之间的相互作用和有序性；
2. 分析系统的相变过程，例如液体 - 固体相变、液体 - 气体相变等；
3. 了解系统的局部结构，例如液体中的簇形成、气体中的原子或分子分布规律等；
4. 与实验数据（如 X 射线衍射、中子散射等）进行对比，验证模拟结果的准确性。

因此，在分子动力学模拟中计算径向分布函数对于分析和解释模拟结果具有重要意义。
</details>

对于 licl.rdf 文件，我们可以利用如下 Python 脚本进行进一步处理和绘图：

In [7]:
# 以下命令判断该环境中是否存在所需模块，如果没有，则使用 pip 快速安装。
! if ! pip show matplotlib > /dev/null; then pip install matplotlib; fi;

In [18]:
import numpy as np
import matplotlib.pyplot as plt

nbins = 100 # define the number of bins in the RDF

with open("./LiCl_DP_Tutorial_Example/chapter1/licl.rdf", "r") as f: # read the licl.rdf file
    lines = f.readlines()
    lines = lines[3:]

    data = np.zeros((nbins, 7))  
    count = 0  

    for line in lines:  
        nums = line.split()      
        if len(nums) == 8:  
            for i in range(1, 8):  
                data[int(nums[0])-1, i-1] += float(nums[i])  # accumulatie data for each bin  
        if len(nums) == 2:  
            count += 1                                       # count the number of accumulations for each bin
       
ave_rdf = data / count                                       # calculate the averaged RDF data
np.savetxt('./LiCl_DP_Tutorial_Example/chapter1/ave_rdf.txt', ave_rdf)

labels = ['Li-Li', 'Li-Cl', 'Cl-Cl']
colors = ['orange', 'purple', 'green']
for i, label, color in zip(range(1, 7, 2), labels, colors):
    plt.plot(ave_rdf[:, 0], ave_rdf[:, i], color=color, label=label)
plt.title('RDF')
plt.xlabel('r/Å')
plt.ylabel('g(r)')
plt.legend()
plt.savefig('./LiCl_DP_Tutorial_Example/chapter1/rdf.png', dpi=300)
plt.show()


<Figure size 640x480 with 1 Axes>

对于 licl.msd 文件，我们可以利用如下 Python 脚本进行进一步处理计算得到扩散系数 D 和绘图：

<details>
<summary>
<b>我们如何通过均方位移（MSD）来计算扩散系数呢？</b>
</summary>
<br/>
我们可以通过均方位移（MSD）计算扩散系数，因为它们之间存在直接的数学关系。扩散系数是一个衡量粒子在物质中传播的速度的物理量，它与粒子的均方位移（MSD）相关。

在扩散过程中，粒子的运动符合随机游走或布朗运动模型。对于三维随机游走，粒子在每个方向上的位移是独立的，我们可以将总的均方位移表示为三个方向的位移平方和的平均值：

$$
\operatorname{MSD}(t)=\left\langle\left|\Delta r_i(t)\right|^2\right\rangle=\frac{1}{N}\left\langle\sum_i\left|r_i(t)-r_i(0)\right|^2\right\rangle
$$

根据爱因斯坦关系（Einstein Relation）和菲克定律（Fick's Law），我们可以得到扩散系数（D）和均方位移之间的关系：

$$
D=\frac{1}{6} \lim _{t \rightarrow \infty} \frac{\mathrm{d}(\text { MSD })}{\mathrm{d} t}
$$

在这个公式中，$t$ 是时间，$D$ 是扩散系数。公式表明，MSD 随时间线性增长，其斜率与扩散系数成正比。因此，我们可以通过对 MSD 数据进行线性拟合，计算出 MSD 与时间的线性关系，从而获得扩散系数。

需要注意的是，这种方法基于扩散过程中粒子的运动是随机的、无记忆的假设。在某些情况下，这种假设可能不成立，比如在有长程相互作用或有序晶格中的粒子运动。在这种情况下，我们需要采用其他方法来计算扩散系数。然而，在许多实际应用中，尤其是液体和气体中的粒子扩散，使用 MSD 计算扩散系数是一种非常有效的方法。
</details>

In [22]:
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt('./LiCl_DP_Tutorial_Example/chapter1/licl.msd', skiprows=2)

time = data[:, 0]
msd1 = data[:, 1]
msd2 = data[:, 2]

plt.plot(time/1000, msd1, 'b-', label='$Li^+$') # 1fs= 1/1000ps
plt.plot(time/1000, msd2, 'r-', label='$Cl^-$')
plt.xlabel('time(ps)') 
plt.ylabel('$MSD(A^2)$')
plt.title('msd')
plt.legend()
plt.savefig('./LiCl_DP_Tutorial_Example/chapter1/msd.png', dpi=300)
plt.show()

# 计算扩散系数
slope1, residuals = np.polyfit(time, msd1, 1)  # 使用 numpy 的 polyfit 函数对 Li+ 和 Cl- 离子的均方位移数据进行一阶多项式（线性）拟合得到斜率
slope2, residuals = np.polyfit(time, msd2, 1)  # 这里，我们假设 MSD 与时间成线性关系，即 MSD(t) = 6*D*t，其中 D 是扩散系数。

Diff1 = slope1/6 * 1e-5  # D=1/6*slope; 1 A^2/fs = 1e-5 m^2/s
Diff2 = slope2/6 * 1e-5

print(f"Diffusion Coefficients of Li+: {Diff1} m^2/s")
print(f"Diffusion Coefficients of Cl-: {Diff2} m^2/s")

<Figure size 640x480 with 1 Axes>

Diffusion Coefficients of Li+: 7.543639618987788e-09 m^2/s
Diffusion Coefficients of Cl-: 3.5602726197335397e-09 m^2/s


以及从均方位移导出的扩散系数的数值：
```
Diffusion Coefficients of Li+: 7.543639618987788e-09 m^2/s
Diffusion Coefficients of Cl-: 3.5602726197335397e-09 m^2/s
```

注意：扩散系数的值可能和[文献](https://doi.org/10.1021/jp5050332)报道存在差异。我们可以考虑以下几个方面，以获得更准确的值：

- 首先在 NPT 系综进行模拟，将压力固定在 0 GPa，获得平衡体积。然后在平衡体积下进行 NVT 模拟。
- 使用更大的模拟盒子。
- 设置更长的模拟时间。

## 章节 2：ABACUS 第一性原理计算

对于 LiCl 熔体，我们还可以使用 ABACUS (Atomic-orbital Based Ab-initio Computation at UStc) 软件进行第一性原理计算。ABACUS 是一个基于密度泛函理论（DFT）的材料模拟软件，可以准确地计算原子间相互作用。通过使用 ABACUS，我们可以从电子结构层面研究 LiCl 熔体的性质。

### 本章节目标

在学习本章节后，你能够：

- 熟悉 ABACUS 软件的输入和输出文件；
- 为 LiCl 熔体编写 ABACUS SCF 计算输入文件并进行计算；
- 为 LiCl 熔体编写 ABACUS MD 计算输入文件并进行计算；

### 2.1 下载教程资源

在本章节中，我们以 LiCl 熔体分子为例,进行 ABACUS 第一性原理计算。我们已经在 *LiCl_DP_Tutorial_Example/chapter2* 中准备了需要的文件，本教程采用 ABACUS v3.1.0 完成。

在 *LiCl_DP_Tutorial_Example/chapter2* 文件夹中存在如下 5 种文件：

- `INPUT`：包含了计算过程中所需的各种参数，定义和控制计算任务；
- `STRU`：结构文件，包含了原子种类、原子位置、晶格常数以及晶格向量等信息；
- `KPT`：包含了布里渊区积分所需的k点信息；
- `*.upf`：包含了原子的赝势信息；
- `*.orb`：包含了原子轨道的数值表示；

### 2.2 ABACUS 自洽计算

首先，我们将使用 ABACUS 软件为 LiCl 熔体执行自洽计算。

自洽，是自洽场计算(SCF, self-consistent field calculation)的简称，它是密度泛函理论（DFT）算法的基础，也是 DFT 被称为第一性原理方法的原因。

从自洽场计算中我们可以得到一个体系的**基态结构**和**基态能量**。

基态结构使得我们可以得到体系的结构参数，例如晶胞参数，化学键键长键角等，而基态能量使得我们可以进一步得到原子受力和晶体材料的应力-应变性质等。

因此自洽场计算是开始做 DFT 计算的基础。

#### 2.2.1 ABACUS 自洽计算输入文件

你刚才下载了我们为本案例准备的 ABACUS 输入文件，让我们来通过以下命令查看。

1. *INPUT* 文件。INPUT 文件包含了与 LiCl 熔体 SCF 计算的相关参数：

In [24]:
! cd ./LiCl_DP_Tutorial_Example/chapter2/abacus_scf && cat INPUT

INPUT_PARAMETERS

#Parameters (1.General)
ntype                   2
symmetry                0
vdw_method              d3_bj

#Parameters (2.Iteration)
ecutwfc                 100
scf_thr                 1e-7
scf_nmax                120

#Parameters (3.Basis)
basis_type              lcao

#Parameters (4.Smearing)
smearing_method         gauss
smearing_sigma          0.002

#Parameters (5.Mixing)
mixing_type             pulay
mixing_beta             0.4

#Parameters (6.sfc)
calculation             scf
cal_force               1
cal_stress              1

对 INPUT 中各参数的介绍如下：

```
INPUT_PARAMETERS

#Parameters (1.General)
ntype                   2           # 原子种类的数量，这里设置为 2
symmetry                0           # 是否使用晶体对称性，0 表示不使用
vdw_method              d3_bj       # 范德华力的计算方法，这里选择 D3 方法，使用 Becke-Johnson 阻尼函数（d3_bj）

#Parameters (2.Iteration)
ecutwfc                 100         # 平面波基组截断能，这里设为 100 Ry
scf_thr                 1e-7        # 自洽场收敛阈值，设为 1.0e-7
scf_nmax                120         # 自洽场迭代的最大步数，设为 120

#Parameters (3.Basis)
basis_type              lcao        # 基组类型，这里选择线性组合原子轨道（LCAO）                     

#Parameters (4.Smearing)
smearing_method         gauss       # 展宽方法，这里选择高斯展宽（gauss）
smearing_sigma          0.002       # 展宽参数，这里设为 0.002 Ry

#Parameters (5.Mixing)
mixing_type             pulay       # 电荷密度混合方法，这里选择 Pulay 混合
mixing_beta             0.4         # 混合参数，设为 0.4

#Parameters (6.sfc)
calculation             scf                
cal_force               1           # 是否计算原子受力，设为 1 表示计算
cal_stress              1           # 是否计算应力，设为 1 表示计算
```

2. *STRU* 文件。STRU 文件包含了 LiCl 熔体的原子种类、原子位置、晶格常数以及晶格向量等信息。考虑到 DFT 计算成本相对较高，这里设计了一个较小的计算体系（64原子）：

In [13]:
! cd ./LiCl_DP_Tutorial_Example/chapter2/abacus_scf && cat STRU

ATOMIC_SPECIES
Li 6.941 Li_ONCV_PBE-1.2.upf
Cl 35.453 Cl_ONCV_PBE-1.2.upf

NUMERICAL_ORBITAL
Li_gga_8au_100Ry_4s1p.orb
Cl_gga_8au_100Ry_2s2p1d.orb

LATTICE_CONSTANT
1.8897261246257702

LATTICE_VECTORS
11.858 0.0 0.0 
0.0 11.858 0.0 
0.0 0.0 11.858 

ATOMIC_POSITIONS
Cartesian    # Cartesian(Unit is LATTICE_CONSTANT)
Li
0.0
32
3.01682  1.77597  4.37873  1  1  1
3.47660  5.23243  2.05957  1  1  1
0.90851  3.90744  4.56133  1  1  1
1.45547  -0.05347  0.89671  1  1  1
9.58599  4.38419  1.31575  1  1  1
5.33635  11.08600  4.46604  1  1  1
7.53108  3.19151  10.88540  1  1  1
6.20144  2.27490  2.29226  1  1  1
10.86000  11.28090  4.03153  1  1  1
8.72296  1.91404  5.01236  1  1  1
0.56714  5.94286  11.29220  1  1  1
4.56436  1.51683  10.93970  1  1  1
6.47417  6.48432  2.26679  1  1  1
2.67649  9.86711  5.62938  1  1  1
10.52500  6.89521  4.55821  1  1  1
7.31791  10.82390  10.75460  1  1  1
9.98168  8.34648  1.34799  1  1  1
8.55005  9.02160  5.95048  1  1  1
0.22956  2.27927  10.61770  1  1

STRU 文件介绍如下：

```
ATOMIC_SPECIES
Li 6.941 Li_ONCV_PBE-1.2.upf           # 元素，原子质量，使用的赝势文件
Cl 35.453 Cl_ONCV_PBE-1.2.upf

NUMERICAL_ORBITAL
Li_gga_8au_100Ry_4s1p.orb              # 数值轨道文件
Cl_gga_8au_100Ry_2s2p1d.orb

LATTICE_CONSTANT
1.889726                               # 1.889726 Bohr =  1.0 Angstrom

LATTICE_VECTORS
11.858 0.0 0.0                         # 晶格向量
0.0 11.858 0.0 
0.0 0.0 11.858 

ATOMIC_POSITIONS
Cartesian                              # 以笛卡尔坐标表示（Cartesian），单位为晶格常数
Li                                     # 元素名称
0.0                                    # 元素磁性
32                                     # 原子个数
3.01682  1.77597  4.37873  1  1  1     # 每个原子x，y，z方向的坐标和约束条件（1表示允许在该方向上移动，0表示固定）
3.47660  5.23243  2.05957  1  1  1
0.90851  3.90744  4.56133  1  1  1
...（省略）
Cl
0.0
32
2.40263  3.44380  3.15593  1  1  1
4.71432  5.62464  3.99601  1  1  1
5.02722  1.62164  4.70153  1  1  1
...（省略）

```

3. *KPT* 文件。KPT 文件包含了 LiCl 熔体 SCF 计算的 k 点设置：

In [25]:
! cd ./LiCl_DP_Tutorial_Example/chapter2/abacus_scf && cat KPT

K_POINTS
0
Gamma
1 1 1 0 0 0


4. `*.upf` 和 `*.orb` 文件。对于 Li 和 Cl，分别采用 Li_ONCV_PBE-1.2.upf 和 Cl_ONCV_PBE-1.2.upf，以及 Li_gga_8au_100Ry_4s1p.orb 和 Cl_gga_8au_100Ry_2s2p1d.orb。各原子的 upf 和 orb 文件可以从 ABACUS 官网下载。

#### 2.2.2 运行 ABACUS 自洽计算

准备好以上所有输入文件后，我们可以执行 LiCl 熔体的 SCF 计算。例如，使用命令行：

- 在此镜像中，由于 ABACUS 是基于 intel oneapi 的环境安装的，我们运行 abacus 之前需要使用命令
`. /opt/intel/oneapi/setvars.sh` 装载相应的环境。
- 设置 `OMP_NUM_THREADS=1` 使用单线程进行计算。
- `abacus` 为 ABACUS 可执行程序的命令。

In [2]:
# ############### Time Warning: 1 minute ###################
! cd ./LiCl_DP_Tutorial_Example/chapter2/abacus_scf && . /opt/intel/oneapi/setvars.sh && OMP_NUM_THREADS=1 mpirun -np 32 abacus

 
:: initializing oneAPI environment ...
   dash: SH_VERSION = unknown
   args: Using "$@" for setvars.sh arguments: 
:: compiler -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 
                                                                                     
                              ABACUS v3.1                                            

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         

 Sun May 21 16:12:48 2023
 MAKE THE DIR         : OUT.ABACUS/

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### 2.2.3 结果与分析

主要的计算信息被储存在文件 *OUT.ABACUS/running_scf.log* 中

我们可以使用 `cat LiCl_DP_Tutorial_Example/chapter2/OUT.ABACUS/running_scf.log` 查看该文件。


在这里，我们提供一个示例来聚焦于重要内容：

```

                                                                                     
                             WELCOME TO ABACUS v3.0                                  
                                                                                     
               'Atomic-orbital Based Ab-initio Computation at UStc'                  
                                                                                     
                     Website: http://abacus.ustc.edu.cn/                             
                                                                                     
    Version: Parallel, in development
    Processor Number is 16
    Start Time is Fri Mar 17 11:12:54 2023
                                                                                     
 ------------------------------------------------------------------------------------

...（省略）

 LCAO ALGORITHM --------------- ION=   1  ELEC=   1--------------------------------

 Density error is 0.104045211662

       Energy                       Rydberg                            eV
   E_KohnSham                 -1429.2427444                 -19445.845149
     E_Harris                -1430.40301784                 -19461.631479
      E_Fermi              +0.0524155151601               +0.713149669783

...（省略）

 LCAO ALGORITHM --------------- ION=   1  ELEC=  11--------------------------------
 Memory of pvpR : 2.89599609375 MB

 Density error is 7.23978511026e-08

       Energy                       Rydberg                            eV
   E_KohnSham                -1429.41030086                -19448.1248715
     E_Harris                -1429.41030086                -19448.1248715
       E_band                -310.415111989                -4223.41426836
   E_one_elec                -849.674984163                -11560.4212327
    E_Hartree                +459.864819535                +6256.78185542
         E_xc                -294.061400137                -4000.91060373
      E_Ewald                -744.876355072                -10134.5627344
      E_demet            -3.01381081966e-88            -4.10049998414e-87
      E_descf                            +0                            +0
      E_vdwD3                -0.66238101901                -9.01215610558
        E_exx                            +0                            +0
      E_Fermi              +0.0508640615939                 +0.6920410611

 charge density convergence is achieved
 final etot is -19448.1248715 eV

...（省略）

```

可以看到，经过 11 次迭代后，电荷密度收敛，密度误差达到 7.23978511026e-08，最终总能量为 -19448.1248715 eV。

**在这个练习中，我们熟悉了 ABACUS 软件的输入文件，学会了如何为 LiCl 熔体编写 ABACUS SCF计算输入文件，执行计算并查看计算收敛情况。**

### 2.3 ABACUS 量子分子动力学计算

量子分子动力学（Quantum Molecular Dynamics，QMD）：将量子力学的原理应用于分子动力学中，考虑电子的波动性和粒子性。

除了 1.4 与 2.3 中提到的两种分子动力学外，还有如：

- 蒙特卡罗分子动力学（Monte Carlo Molecular Dynamics，MCMD）：使用随机模拟的方法来模拟分子的运动，通过统计模拟得到分子系统的状态。

- 多尺度分子动力学（Multiscale Molecular Dynamics，MMD）：将不同尺度的分子动力学方法结合起来，以提高模拟效率和准确性。

- 拉格朗日分子动力学（Lagrangian Molecular Dynamics，LMD）：采用拉格朗日力学的方法来描述分子系统的运动。

- **基于机器学习的分子动力学（Machine Learning Molecular Dynamics，MLMD）：利用机器学习方法来辅助分子动力学模拟，以提高模拟效率和准确性。**

在这篇教程中即将学习的 DeePMD-kit 即为基于机器学习的分子动力学。

#### 2.3.1 ABACUS MD 计算输入文件

ABACUS MD 计算所需的输入文件与上述 ABACUS SCF 计算的输入文件相似。

我们仅需要修改 INPUT 文件，而其他文件无需更改。修改后的 INPUT 文件示例如下：

```
INPUT_PARAMETERS

#Parameters (1.General)
ntype           2              
symmetry        0                      
vdw_method      d3_bj          

#Parameters (2.Iteration)
ecutwfc         100            
scf_thr         1.0e-7         
scf_nmax        120            

#Parameters (3.Basis)        
basis_type      lcao           
          
#Parameters (4.Smearing)
smearing_method	gauss          
smearing_sigma	0.002          

#Parameters (5.Mixing)
mixing_type	pulay          
mixing_beta	0.4            

#Parameters (6.md)
calculation     md             # calculation：计算类型，这里选择分子动力学（md）
cal_force       1              
cal_stress      1              
md_nstep        500            # md_nstep：分子动力学模拟的总步数，设为 500
md_type         1              # md_type：模拟类型，这里选择 NVT 系综
md_dt           1              # md_dt：时间步长，设为 1
md_tfirst       900            # md_tfirst：目标温度，设为 900 K
md_restart      0              # md_restart：是否为续算，设为 0 表示不是续算
md_dumpfreq     1              # md_dumpfreq：输出分子动力学信息的频率，设为 1 表示每一步都输出
out_stru        1              # out_stru：输出结构信息的选项，设为 1 表示输出
```
INPUT 文件定义了 LiCl 熔体的分子动力学（MD）模拟。与之前的 LAMMPS MD 一致，模拟在 NVT 系综下进行，时间步长 $1 fs$，温度为 900 K。

在 *LiCl_DP_Tutorial_Example/chapter2/abacus_md/* 文件夹中我们为你准备好了本次计算所需的输入文件。

考虑到 DFT 计算成本相对较高，这里模拟时间为 $0.5 ps$ ($1 ps = 1000 fs = 10^{-12} s$)。

注意：使用 LCAO 基组计算时，通常需测试能量、力、应力对不同轨道半径截断值和不同 k 点（k_spacing）设置的收敛性。出于简化教程的目的，这里我们没有做这样的测试。

#### 2.3.2 运行 ABACUS MD 计算

准备好以上所有输入文件后，我们可以执行 LiCl 熔体的MD计算。例如，使用命令行：

**注意：** 由于我们需要的 ABACUS 版本为 ABACUS 3.1，这一步需将镜像更换为`abacus:3.1.4-toolkit-notebook`。

In [ ]:
# ############ Time Warning: 9 h 30 mins 58 secs ############
# 该命令的输出极长，你可以右键单击输出内容，选择滚动模式查看输出，或直接点击左侧蓝条隐藏输出。
! cd ./LiCl_DP_Tutorial_Example/chapter2/abacus_md && . /opt/intel/oneapi/setvars.sh && OMP_NUM_THREADS=1 mpirun -np 32 abacus

 
:: initializing oneAPI environment ...
   dash: SH_VERSION = unknown
   args: Using "$@" for setvars.sh arguments: 
:: compiler -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 
                                                                                     
                              ABACUS v3.1                                            

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         

 Sun May 21 16:17:00 2023
 MAKE THE DIR         : OUT.ABACUS/
 MAKE THE STRU DIR    : OUT.ABACUS/

可以看到，即使**计算步数仅为** LAMMPS 经典分子动力学计算的 $500/200000 = 1/400$，

ABACUS 量子分子动力学计算所需要的时间也是 LAMMPS 经典分子动力学计算的$9 h 30 mins 58 secs \div 3 min 35 secs = 159.33$ 倍！

成本比高达约 63732 倍！

**第一性原理计算十分昂贵！**

#### 2.3.3 结果与分析

上面已经介绍，主要的计算信息被储存在文件 *OUT.ABACUS/running_scf.log* 中。除此之外，MD 模拟的轨迹被存储在文件 *OUT.ABACUS/MD_dump* 中。

*MD_dump* 文件内容如下：
```
MDSTEP:  0
LATTICE_CONSTANT: 1.889726124626
LATTICE_VECTORS
  11.858000000000  0.000000000000  0.000000000000
  0.000000000000  11.858000000000  0.000000000000
  0.000000000000  0.000000000000  11.858000000000
VIRIAL (KBAR)
  14.314395848598  0.068950391132  -1.646206932245
  0.068950391132  6.742623812696  -3.705349135941
  -1.646206932245  -3.705349135941  5.270646592369
INDEX    LABEL    POSITIONS    FORCE (eV/Angstrom)
  0  Li  3.016819999993  1.775969999997  4.378729999996  -0.812213164484  -0.585472450641  0.311819994732
  1  Li  3.476600000010  5.232430000004  2.059570000010  0.352252495274  -0.175519926585  0.080617217948
  2  Li  0.908510000006  3.907440000006  4.561330000000  -0.649106916034  0.039426589767  0.878683371838
  ...
  61  Cl  10.362100000004  9.238389999995  8.344720000004  -2.421125883128  0.103626611523  -1.137264969726
  62  Cl  9.088940000007  5.618780000000  9.584660000001  -0.546302523476  -2.531252034211  0.628161793135
  63  Cl  8.152059999999  7.237340000007  6.577169999999  -0.867749328458  -2.567100713260  0.810628619279
```


该文件记录了分子动力学模拟的信息。包括：
- MD的步数（MDSTEP）
- 晶格常数（LATTICE_CONSTANT）和晶格矢量（LATTICE_VECTORS）。
- 应力张量（VIRIAL），单位 KBAR。
- 每个原子的索引、标签、坐标位置（POSITIONS）和受到的力（FORCE），力的单位 eV/Angstrom。

这里提供了一个 Python 脚本，可以提取原子坐标，计算 RDF 并将 RDF 数据保存到名为 *gr_Li-Cl*、*gr_Li-Li* 和 *gr_Cl-Cl* 文件。


In [12]:
import numpy as np
import matplotlib.pyplot as plt

def get_atom_positions(file_path, nLi, nCl):
    Li, Cl = [], []
    with open(file_path, 'r') as file:
        for line in file:
            if 'Li' in line:
                Li.append([float(line.split()[2]), float(line.split()[3]), float(line.split()[4])])
            if 'Cl' in line:
                Cl.append([float(line.split()[2]), float(line.split()[3]), float(line.split()[4])])
    return np.array(Li).reshape(-1, nLi, 3), np.array(Cl).reshape(-1, nCl, 3)

def get_gr(x, y, L): 
    batchsize, n, dim = x.shape[0], x.shape[1], x.shape[2]
    
    i,j = np.triu_indices(n, k=1)
    rij = (np.reshape(x, (-1, n, 1, dim)) - np.reshape(y, (-1, 1, n, dim)))[:,i,j]
    rij = rij - L*np.rint(rij/L)
    dist = np.linalg.norm(rij, axis=-1) # (batchsize, n*(n-1)/2)
   
    hist, bin_edges = np.histogram(dist.reshape(-1,), range=[0, L/2], bins=200)
    dr = bin_edges[1] - bin_edges[0]
    hist = hist*2/(n * batchsize)

    rmesh = np.arange(hist.shape[0])*dr
    
    h_id = 4/3*np.pi*n/(L**3)* ((rmesh+dr)**3 - rmesh**3 )
    return rmesh, hist/h_id

L = 11.858
nLi, nCl = 32, 32
Li, Cl = get_atom_positions('./LiCl_DP_Tutorial_Example/chapter2/abacus_md/OUT.ABACUS/MD_dump', nLi, nCl)

atom_pairs = {'Li-Cl': (Li, Cl),'Li-Li': (Li, Li),'Cl-Cl': (Cl, Cl)}

plt.figure(figsize=(10, 8))
colors = ['purple', 'orange', 'green']
for (label, (x, y)), color in zip(atom_pairs.items(), colors):
    rmesh, gr = get_gr(x, y, L)
    plt.plot(rmesh, gr, label=label, color=color)
    np.savetxt(f'./LiCl_DP_Tutorial_Example/chapter2/abacus_md/OUT.ABACUS/gr_{label}', np.column_stack((rmesh, gr)))
    
plt.legend()
plt.title('abacus_md_rdf')
plt.savefig('./LiCl_DP_Tutorial_Example/chapter2/abacus_md/abacus_md_rdf.png', dpi=300)
plt.show()

<Figure size 1000x800 with 1 Axes>

从 ABACUS MD计算的 RDF 如图。我们可以观察 ABACUS 和 LAMMPS MD 获得的 900K 下的 LiCl 熔体的 RDF 是否存在差异。注意，由于模拟体系较小和模拟时间较短，ABACUS MD 计算的 RDF 曲线并不光滑，更大的模拟体系或者更长的模拟时间将有助于改善这一情况。

**在这个练习中，我们学习了如何为 LiCl 熔体编写 ABACUS MD 计算输入文件，执行计算，和 MD 轨迹处理。**

## 章节 3：DeePMD-kit 深度势能 (DP) 模型训练

对于 LiCl 熔体，我们可以使用 DeePMD-kit 软件包，为其训练一个深度势能模型。DeePMD-kit 是一个基于深度学习的分子动力学模拟工具，可以根据第一性原理数据训练高精度的 DP 模型。在这个示例中，我们可以利用 ABACUS 第一性原理数据和 DeePMD-kit 训练一个 LiCl 熔体的 深度势能 (DP) 模型。

DeePMD-kit 的详细原理是：通过建立深度学习模型来学习高成本第一性原理计算获得的数据，得到的深度势能模型，并将该模型应用于后续的分子动力学模型中，用于控制分子间势，这即是「深度势能分子动力学」名称的由来。

DeePMD-kit 深度势能分子动力学方法大大节省了量子分子动力学所需的计算成本，将分子动力学模拟的规模从以往的几千原子提升至上亿原子的级别，同时也保留了显著优于经典分子动力学的模拟精度。相关成果曾获2020年度⾼性能计算领域最⾼奖 ACM 戈登贝尔奖。

### 本章节目标

学习完本章节后，你将能够：

- 掌握 DeePMD-kit 输入文件编写
- 能够进行数据准备、训练/冻结/压缩/测试和分子动力学任务



### 下载教程资源

在本章节中，我们以 LiCl 熔体分子为例,训练深度势能模型。我们已经在 *./LiCl_DP_Tutorial_Example/chapter3* 中准备了需要的文件。

在 *./LiCl_DP_Tutorial_Example/chapter3* 文件夹下有 00.data，01.train 和 02.lmp 共 3 个子文件夹。

- *00.data* 文件夹用于存放训练和测试数据，
- *01.train* 包含使用 DeePMD-kit 训练模型的示例脚本，
- *02.lmp* 包含用于分子动力学模拟的 LAMMPS 示例脚本。

本教程采用 DeePMD-kit(2.2.1)程序完成。


### 3.1 准备训练数据

在 2.3 中已经执行了ABACUS MD 计算。你可以使用如下 Python 脚本，其中调用 [dpdata](https://github.com/deepmodeling/dpdata) 的工具，将 ABACUS MD 生成的数据（数据格式abacus/md）转换为 DeePMD-kit 训练所需的数据格式（NumPy数组）。

In [26]:
import dpdata 
import numpy as np

# 加载 abacus/md 格式数据
data = dpdata.LabeledSystem('./LiCl_DP_Tutorial_Example/chapter2/abacus_md', fmt = 'abacus/md')        

# 随机选择100个索引，用于生成验证集;其他的索引，用于生成测试集
index_validation = np.random.choice(len(data),size=100,replace=False)    
index_training = list(set(range(len(data)))-set(index_validation)) 

# 创建子数据集：训练集,测试集      
data_training = data.sub_system(index_training)                          
data_validation = data.sub_system(index_validation)

# 导出训练集,测试集（deepmd/npy格式）                     
data_training.to_deepmd_npy('./LiCl_DP_Tutorial_Example/chapter3/00.data/training_data')                                                       
data_validation.to_deepmd_npy('./LiCl_DP_Tutorial_Example/chapter3/00.data/validation_data')                         

print('# 数据包含 %d frames' % len(data))     
print('# 训练数据包含 %d frames' % len(data_training)) 
print('# 验证数据包含 %d frames' % len(data_validation))

# 数据包含 501 frames
# 训练数据包含 401 frames
# 验证数据包含 100 frames


我们可以看到 *abacus_md* 文件包含 501 帧数据。 我们随机选取 100 帧作为验证数据，其余的401帧作为训练数据。在开始训练之前，我们可以先检查 *training_data* 或 *validation_data* 文件夹。

In [16]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/00.data/ && ls training_data

box.raw    energy.raw  set.000	 type_map.raw
coord.raw  force.raw   type.raw  virial.raw


这些文件的作用如下：

1. `set.000`：是一个目录，包含压缩格式的数据（NumPy压缩数组）。所有训练数据应首先转换为此格式，然后在 DeePMD-kit 中使用。该数据格式在 DeePMD-kit 手册中有详细解释，可以在 [DeePMD-kit Data Introduction](https://github.com/deepmodeling/deepmd-kit/blob/edb4e6951f3300b8669e73fb34832f90c02c25f3/doc/data/index.md) 中找到。
2. `type.raw`：是一个文件，包含原子的类型（以整数表示）。
3. `type_map.raw`：是一个文件，包含原子的类型名称。

让我们来看一下这些文件：

In [27]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/00.data/ && cat training_data/type.raw 

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


这告诉我们这个例子中有 108 个原子，其中 54 个原子由类型"0"表示，54 个原子由类型"1"表示。有时需要将整数类型映射到原子名称。映射可以通过文件`type_map.raw`给出。

由于系统中的所有帧都具有相同的原子类型和原子序号，因此我们只需为整个系统指定一次类型信息。

In [28]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/00.data/ && cat training_data/type_map.raw

Li
Cl


其中原子 Li 被赋予类型 0，原子 Cl 被赋予类型 1。

### 3.2 准备输入脚本

训练数据准备完成后，接下来就可以进行训练。让我们进入训练目录看一下运行所需的脚本：

In [29]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/01.train/ && cat input.json

{
    "_comment": " model parameters",
    "model": {
	"type_map":	["Li", "Cl"],
	"descriptor" :{
	    "type":		"se_e2_a",
	    "sel":		[128, 128],
	    "rcut_smth":	0.50,
	    "rcut":		7.00,
	    "neuron":		[20, 40, 80],
	    "resnet_dt":	false,
	    "axis_neuron":	4,
	    "seed":		1,
	    "_comment":		" that's all"
	},
	"fitting_net" : {
	    "neuron":		[200, 200, 200],
	    "resnet_dt":	true,
	    "seed":		1,
	    "_comment":		" that's all"
	},
	"_comment":	" that's all"
    },

    "learning_rate" :{
	"type":		"exp",
	"decay_steps":	5000,
	"start_lr":	0.001,	
	"stop_lr":	3.51e-8,
	"_comment":	"that's all"
    },

    "loss" :{
	"type":		"ener",
	"start_pref_e":	0.02,
	"limit_pref_e":	1,
	"start_pref_f":	1000,
	"limit_pref_f":	1,
	"start_pref_v":	0,
	"limit_pref_v":	0,
	"_comment":	" that's all"
    },

    "training" : {
	"training_data": {
	    "systems":     ["../00.data/training_data"],
	    "batch_size":  "auto",
	    "_comment":	   "that's all"
	},
	"validation_data":{
	    "s


input.json 包含了 DP 模型训练过程中所需的各种参数，定义和控制训练任务。这些参数在 DeePMD-kit 手册中有详细的解释，所以这里只做简单介绍。

在 model 模块, 指定嵌入和拟合网络的参数。
```
    "model":{
    "type_map":    ["Li", "Cl"],                         # 元素名称
    "descriptor":{
        "type":            "se_e2_a",                    # 描述符类型
        "rcut":            7.00,                         # 截止半径
        "rcut_smth":       0.50,                         # 光滑截止半径
        "sel":             [128, 128],                   # 原子的选定邻居数
        "neuron":          [20, 40, 80],                 # 嵌入网络尺寸
        "resnet_dt":       false,
        "axis_neuron":     12,                           # 嵌入子网络横向尺寸
        "seed":            1,
        "_comment":        "that's all"
    },
    "fitting_net":{
        "neuron":          [200, 200, 200],              # 拟合网络尺寸
        "resnet_dt":       true,
        "seed":            1,
        "_comment":        "that's all"
    },
    "_comment":    "that's all"'
},
```
描述符 `se_e2_a` 用于 DP 模型的训练。将嵌入和拟合神经网络的大小分别设置为 [20, 40, 80] 和 [200, 200, 200]。 $\tilde{\mathcal{R}}^{i}$里的成分会从 0.5 到 7 Å 平滑地趋于 0。

下面的参数指定学习效率和损失函数：
```
    "learning_rate" :{
        "type":                "exp",
        "decay_steps":         5000,                      # 学习率下降间隔
        "start_lr":            0.001,                     # 起始学习率   
        "stop_lr":             3.51e-8,                   # 结束学习率 
        "_comment":            "that's all"
    },
    "loss" :{
        "type":                "ener",
        "start_pref_e":        0.02,                      # 能量起始权重
        "limit_pref_e":        1,                         # 能量最终权重
        "start_pref_f":        1000,                      # 力起始权重
        "limit_pref_f":        1,                         # 力最终权重
        "start_pref_v":        0,                         # 维里
        "limit_pref_v":        0,
        "_comment":            "that's all"
    },
```

在损失函数中，${pref\_e}$ 从 0.02 逐渐增加到 1 ${eV}^{-2}$，而 ${pref\_f}$ 从 1000 逐渐减小到 1 ${Å}^2 {eV}^{-2}$，这意味着力项在开始时占主导地位，而能量项和维里项在结束时变得重要。这种策略非常有效，并且减少了总训练时间。${pref\_v}$ 设为 0 ${eV}^{-2}$，这表明训练过程中不包含任何维里数据。将起始学习率、停止学习率和衰减步长分别设置为 0.001，3.51e-8，和 5000。模型训练步数为 $10^6$。


训练参数如下：
```
    "training" : {
        "training_data": {
            "systems":            ["../00.data/training_data"],         # 训练数据路径
            "batch_size":         "auto",                               # 自动确定，natoms*batch_size≥32
            "_comment":           "that's all"
        },
        "validation_data":{
            "systems":            ["../00.data/validation_data/"],
            "batch_size":         "auto",				
            "numb_btch":          1,                                    # 测试帧数
            "_comment":           "that's all"
        },
        "numb_steps":             400000,                               # 训练步数
        "seed":                   10,
        "disp_file":              "lcurve.out",                         # 写入学习曲线到文件
        "disp_freq":              1000,                                 # 写入学习曲线的频率
        "save_freq":              10000,                                # 保存模型相关文件频率
    },
```


### 3.3 模型训练

准备好训练脚本后，我们可以用 DeePMD-kit 开始训练，只需运行:

In [ ]:
# ############## Time Warning: 7 hours ##################
! cd ./LiCl_DP_Tutorial_Example/chapter3/01.train/ && dp train input.json


在 train.log 可以看到数据系统的信息
```
DEEPMD INFO      ----------------------------------------------------------------------------------------------------
DEEPMD INFO      ---Summary of DataSystem: training     -------------------------------------------------------------
DEEPMD INFO      found 1 system(s):
DEEPMD INFO                              system        natoms        bch_sz        n_bch          prob        pbc
DEEPMD INFO           ../00.data/training_data/            64             1          401         1.000          T
DEEPMD INFO      -----------------------------------------------------------------------------------------------------
DEEPMD INFO      ---Summary of DataSystem: validation   --------------------------------------------------------------
DEEPMD INFO      found 1 system(s):
DEEPMD INFO                               system       natoms        bch_sz        n_bch          prob        pbc
DEEPMD INFO          ../00.data/validation_data/           64             1          100         1.000          T
```


以及本次训练的开始和最终学习率
```
DEEPMD INFO      start training at lr 1.00e-03 (== 1.00e-03), decay_step 5000, decay_rate 0.950006, final lr will be 3.51e-08
```


如果一切正常，将在屏幕上看到每 1000 步打印一次的信息，例如
```
DEEPMD INFO    batch    1000 training time 69.23 s, testing time 0.06 s
DEEPMD INFO    batch    2000 training time 68.00 s, testing time 0.06 s
DEEPMD INFO    batch    3000 training time 67.80 s, testing time 0.06 s
DEEPMD INFO    batch    4000 training time 69.92 s, testing time 0.07 s
DEEPMD INFO    batch    5000 training time 67.98 s, testing time 0.06 s
```
 在第 10000 步结束时，模型保存在 TensorFlow 的检查点文件 model.ckpt 中。 同时，训练和测试错误显示在文件 lcurve.out 中。


In [19]:
! head -n 3 ./LiCl_DP_Tutorial_Example/chapter3/01.train/lcurve.out && tail -n 2 ./LiCl_DP_Tutorial_Example/chapter3/01.train/lcurve.out

'''
#  step      rmse_val    rmse_trn    rmse_e_val  rmse_e_trn    rmse_f_val  rmse_f_trn         lr
      0      1.41e+01    1.27e+01      3.50e-01    3.35e-01      4.45e-01    4.02e-01    1.0e-03
...
 399000      1.57e-02    1.55e-02      8.17e-05    4.88e-04      1.53e-02    1.47e-02    4.0e-08
 400000      1.55e-02    1.54e-02      2.68e-04    4.64e-04      1.51e-02    1.47e-02    3.5e-08
'''


#  step      rmse_val    rmse_trn    rmse_e_val  rmse_e_trn    rmse_f_val  rmse_f_trn         lr
      0      1.59e+01    1.52e+01      3.66e-01    3.41e-01      5.01e-01    4.80e-01    1.0e-03
   1000      1.60e+00    1.51e+00      5.06e-02    4.82e-02      5.07e-02    4.78e-02    1.0e-03
 399000      1.44e-02    1.61e-02      1.82e-04    4.83e-05      1.41e-02    1.58e-02    4.0e-08
 400000      1.61e-02    1.44e-02      5.06e-04    5.25e-04      1.53e-02    1.36e-02    3.5e-08


'\n#  step      rmse_val    rmse_trn    rmse_e_val  rmse_e_trn    rmse_f_val  rmse_f_trn         lr\n      0      1.41e+01    1.27e+01      3.50e-01    3.35e-01      4.45e-01    4.02e-01    1.0e-03\n...\n 399000      1.57e-02    1.55e-02      8.17e-05    4.88e-04      1.53e-02    1.47e-02    4.0e-08\n 400000      1.55e-02    1.54e-02      2.68e-04    4.64e-04      1.51e-02    1.47e-02    3.5e-08\n'

第 4、5 和 6、7 列分别介绍了能量和力的训练和测试误差。 经过 4000,00 步训练，能量测试误差小于 1 meV，力测试误差小于 20 meV/Å。可以通过简单的 Python 脚本对该文件进行可视化：


In [20]:
import numpy as np
import matplotlib.pyplot as plt

data = np.genfromtxt("./LiCl_DP_Tutorial_Example/chapter3/01.train/lcurve.out", names=True)
for name in data.dtype.names[1:-1]:
    plt.plot(data['step'], data[name], label=name)
plt.legend()
plt.xlabel('Step')
plt.ylabel('Loss')
plt.yscale('log')
plt.show()


<Figure size 640x480 with 1 Axes>

当训练过程异常停止时，我们可以从提供的检查点重新开始训练，只需运行

In [ ]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/01.train/ && dp train --restart model.ckpt input.json

需要注意的是 *input.json* 需要和上一个保持一致。



### 3.4 冻结和压缩模型

在训练结束时，保存在 TensorFlow 的 checkpoint 文件中的模型参数通常需要冻结为一个以扩展名 .pb 结尾的模型文件。 只需执行：


In [2]:
# 该步骤需要您运行完毕上述训练步骤，产生运行中间文件后方可运行。
! cd ./LiCl_DP_Tutorial_Example/chapter3/01.train/ && dp freeze -o licl.pb

Instructions for updating:
non-resource variables are not supported in the long term
/opt/deepmd-kit-2.2.1/lib/python3.10/importlib/__init__.py:169: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  _bootstrap._exec(spec, module)
2023-04-26 13:04:57.131671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: /usr/lib/x86_64-linux-gnu/libcuda.so.1: file too short; LD_LIBRARY_PATH: /opt/intel/oneapi/tbb/2021.5.1/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/mpi/2021.5.1//libfabric/lib:/opt/intel/oneapi/mpi/2021.5.1//lib/release:/opt/intel/oneapi/mpi/2021.5.1//lib:/opt/intel/oneapi/mkl/2022.0.2/lib/intel64:/opt/intel/oneapi/debugger/2021.5.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2021.5.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2021.5.0/dep/lib:/opt/intel/oneapi/compiler/2022.0.2/linux/lib:/opt/intel/oneapi/compiler/2

它将在当前目录中输出一个名为 graph.pb 的模型文件。 压缩 DP 模型通常会将基于 DP 的计算速度提高一个数量级，并且消耗更少的内存。 licl.pb 可以通过以下方式压缩：


In [3]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/01.train/ && dp compress -i licl.pb -o licl-compress.pb

Instructions for updating:
non-resource variables are not supported in the long term
/opt/deepmd-kit-2.2.1/lib/python3.10/importlib/__init__.py:169: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  _bootstrap._exec(spec, module)
2023-04-26 13:06:05.348719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: /usr/lib/x86_64-linux-gnu/libcuda.so.1: file too short; LD_LIBRARY_PATH: /opt/intel/oneapi/tbb/2021.5.1/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/mpi/2021.5.1//libfabric/lib:/opt/intel/oneapi/mpi/2021.5.1//lib/release:/opt/intel/oneapi/mpi/2021.5.1//lib:/opt/intel/oneapi/mkl/2022.0.2/lib/intel64:/opt/intel/oneapi/debugger/2021.5.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2021.5.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2021.5.0/dep/lib:/opt/intel/oneapi/compiler/2022.0.2/linux/lib:/opt/intel/oneapi/compiler/2

将输出一个名为 licl-compress.pb 的模型文件。



### 3.5 模型测试
我们可以通过运行如下命令检查训练模型的质量


In [4]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/01.train/ && dp test -m licl-compress.pb -s ../00.data/validation_data -n 100 -d results

Instructions for updating:
non-resource variables are not supported in the long term
/opt/deepmd-kit-2.2.1/lib/python3.10/importlib/__init__.py:169: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  _bootstrap._exec(spec, module)
2023-04-26 13:06:37.982105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: /usr/lib/x86_64-linux-gnu/libcuda.so.1: file too short; LD_LIBRARY_PATH: /opt/intel/oneapi/tbb/2021.5.1/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/mpi/2021.5.1//libfabric/lib:/opt/intel/oneapi/mpi/2021.5.1//lib/release:/opt/intel/oneapi/mpi/2021.5.1//lib:/opt/intel/oneapi/mkl/2022.0.2/lib/intel64:/opt/intel/oneapi/debugger/2021.5.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2021.5.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2021.5.0/dep/lib:/opt/intel/oneapi/compiler/2022.0.2/linux/lib:/opt/intel/oneapi/compiler/2

在屏幕上，可以看到验证数据的预测误差信息。

它将在当前目录中输出名为 results.e.out 和 results.f.out 的文件。类似地，可以通过简单的 Python 脚本对该文件进行可视化：


In [5]:
import numpy as np
import matplotlib.pyplot as plt

# 定义绘制散点图和对角线的函数
def plot(ax, data, key, xlabel, ylabel, min_val, max_val):
    data_key = f'data_{key}'
    pred_key = f'pred_{key}'
    ax.scatter(data[data_key], data[pred_key], label=key, s=6)
    ax.legend()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim(min_val, max_val)
    ax.set_ylim(min_val, max_val)
    ax.plot([min_val, max_val], [min_val, max_val], 'r', lw=1)

# 读取数据，并对e数据进行原子化处理
natom = 64
data_e = np.genfromtxt("./LiCl_DP_Tutorial_Example/chapter3/01.train/results.e.out", names=["data_e", "pred_e"])
data_f = np.genfromtxt("./LiCl_DP_Tutorial_Example/chapter3/01.train/results.f.out", names=["data_fx", "data_fy", "data_fz", "pred_fx", "pred_fy", "pred_fz"])

for col in ['data_e', 'pred_e']:
    data_e[col] /= natom

# 计算e和f的最小值和最大值
data_e_stacked = np.column_stack((data_e['data_e'], data_e['pred_e']))
data_f_stacked = np.column_stack((data_f['data_fx'], data_f['data_fy'], data_f['data_fz'], data_f['pred_fx'], data_f['pred_fy'], data_f['pred_fz']))

min_val_e, max_val_e = np.min(data_e_stacked), np.max(data_e_stacked)
min_val_f, max_val_f = np.min(data_f_stacked), np.max(data_f_stacked)

# 绘制散点图并保存结果
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
plot(axs[0], data_e, 'e', 'DFT energy (eV/atom)', 'DP energy (eV/atom)', min_val_e, max_val_e)
for force_direction in ['fx', 'fy', 'fz']:
    plot(axs[1], data_f, force_direction, 'DFT force (eV/Å)', 'DP force (eV/Å)', min_val_f, max_val_f)
plt.show()

<Figure size 1200x500 with 2 Axes>

### 3.6 使用 LAMMPS 运行深度势能分子动力学


首先，我们将 *01.train* 目录中的 DP 模型复制到 *02.lmp* 目录

In [6]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/02.lmp/ && cp ../01.train/licl-compress.pb ./

让我们查看一下 *02.lmp* 目录下的文件：

In [7]:
! cd ./LiCl_DP_Tutorial_Example/chapter3/02.lmp/ && ls

64_dpmd_rdf.png  licl-compress.pb  licl.dump  licl.rdf
ave_rdf.txt	 licl.data	   licl.in    log.lammps



其中 licl.data 给出了 LiCl 熔体 MD 模拟的初始配置，文件 licl.in 是 lammps 输入脚本。 可以检查 licl.in 并发现它是一个用于 MD 模拟的相当标准的 LAMMPS 输入文件，与章节 1 中的 licl.in 文件相比，在原子类型和势函数参数设置上略有不同：

```
atom_style  atomic
```
和

```
pair_style  licl-compress.pb
pair_coeff  * *
```

其中调用 pair style deepmd 并提供模型文件 licl-compress.pb，这意味着原子间相互作用将由名为 licl-compress.pb 的 DP 模型计算。可以以标准方式执行：


In [ ]:
# ###### Time Warning: 19 minutes 16 seconds #####################
! cd ./LiCl_DP_Tutorial_Example/chapter3/02.lmp/ && lmp -i licl.in

稍等片刻，MD 模拟结束，生成 log.lammps 和 licl.dump 文件。 它们分别存储热力学信息和分子的轨迹，我们可以使用 chapter1 中提供的 Python 脚本计算 RDF。

In [13]:
import numpy as np
import matplotlib.pyplot as plt

nbins = 100 # define the number of bins in the RDF

with open("./LiCl_DP_Tutorial_Example/chapter3/02.lmp/licl.rdf", "r") as f: # read the licl.rdf file
    lines = f.readlines()
    lines = lines[3:]

    data = np.zeros((nbins, 7))  
    count = 0  

    for line in lines:  
        nums = line.split()      
        if len(nums) == 8:  
            for i in range(1, 8):  
                data[int(nums[0])-1, i-1] += float(nums[i])  # accumulatie data for each bin  
        if len(nums) == 2:  
            count += 1         # count the number of accumulations for each bin
       
ave_rdf = data / count  # calculate the averaged RDF data
np.savetxt('./LiCl_DP_Tutorial_Example/chapter3/02.lmp/ave_rdf.txt', ave_rdf)

labels = ['Li-Li', 'Li-Cl', 'Cl-Cl']
colors = ['orange', 'purple', 'green']

for i, label, color in zip(range(1, 7, 2), labels, colors):
    plt.plot(ave_rdf[:, 0], ave_rdf[:, i], label=label, color=color)
plt.xlabel('r/Å')
plt.ylabel('g(r)')
plt.legend()
plt.savefig('./LiCl_DP_Tutorial_Example/chapter3/02.lmp/64_dpmd_rdf.png', dpi=300)
plt.show()

<Figure size 640x480 with 1 Axes>

让我们来对比一下由经典分子动力学、量子分子动力学、深度势能分子动力学计算得到的 rdf 曲线：

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# 读取图片
cmd_rdf = mpimg.imread('./LiCl_DP_Tutorial_Example/chapter1/rdf.png')
qmd_rdf = mpimg.imread('./LiCl_DP_Tutorial_Example/chapter2/abacus_md/abacus_md_rdf.png')
dpmd_rdf = mpimg.imread('./LiCl_DP_Tutorial_Example/chapter3/02.lmp/64_dpmd_rdf.png')

# 设置画布大小
fig, axd = plt.subplot_mosaic([['upleft', 'right'],
                               ['lowleft', 'right']], figsize=(10,8), dpi=300)

# 添加子图1
axd['upleft'].set_title('LAMMPS CMD')
axd['upleft'].imshow(cmd_rdf)
axd['upleft'].axis('off')

# 添加子图2
axd['right'].set_title('ABACUS QMD')
axd['right'].imshow(qmd_rdf)
axd['right'].axis('off')

# 添加子图3
axd['lowleft'].set_title('DeePMD-kit DPMD')
axd['lowleft'].imshow(dpmd_rdf)
axd['lowleft'].axis('off')

# 显示图片
plt.savefig('./LiCl_DP_Tutorial_Example/chapter3/02.lmp/all_rdf.png')
plt.show()

<Figure size 3000x2400 with 3 Axes>

可以发现，相对于 LAMMPS 经典分子动力学计算得到的结果，由 DeePMD-kit 深度势能分子动力学计算得到的结果更接近 ABACUS 量子分子动力学得到的结果，具有更高的准确度。

而在模拟步数为量子分子动力学的 1000000/500 = 200 倍的情况下，时间仅为 19 mins / 9 h 30 mins = 0.03。在保证较高精度的前提下，大大降低了计算成本。



接下来，我们将使用 DP-GEN 工作流进一步优化我们的深度势能分子动力学模拟。在更大空间和时间尺度上研究 LiCl 熔体的结构、动力学和热力学性质。

## 章节 4：DP-GEN 构建训练数据并生成深度势模型

为了构建高质量的LiCl熔体深度势能模型的训练数据集，我们可以使用 DP-GEN(Deep Potential GENerator) 软件。DP-GEN 是一个自动化的训练数据集生成工具，可以根据预设的参数和条件搜索材料的结构和相态空间。

本教程采用DeePMD-kit(2.2.1),ABACUS(3.2.0)和DP-GEN(0.11.0)程序完成。

### 本章节目标：

在学习本章节后，你将能够：

- 掌握 DP-GEN 输入文件（param.json 和 machine.json） 中主要关键词的设置；
- 利用 DP-GEN 为 LiCl 熔体构建训练数据并生成深度势模型；
- 分析和解读 DP-GEN 迭代过程中的结果和输出文件，以便更好地理解模型训练的过程和效果。


### 4.1 下载教程资源

在本教程中，我们以 LiCl 熔体分子为例，构建训练数据并生成深度势模型。我们已经在 *LiCl_DP_Tutorial_Example/chapter4* 中准备了需要的文件。

In [3]:
! if ! pip show dpgen > /dev/null; then pip install dpgen; fi;

/usr/bin/sh: 1: pip: not found
/usr/bin/sh: 1: pip: not found


让我们来查看一下 LiCl_DP_Tutorial_Example/chapter4 中所含的文件。

In [23]:
! tree LiCl_DP_Tutorial_Example/chapter4 -L 2

LiCl_DP_Tutorial_Example/chapter4
├── abacus
│   ├── Cl_ONCV_PBE-1.2.upf
│   ├── Cl_gga_8au_100Ry_2s2p1d.orb
│   ├── Li_ONCV_PBE-1.2.upf
│   └── Li_gga_8au_100Ry_4s1p.orb
├── dpgen.log
├── iter.000000
│   └── 00.train
├── machine.json
├── max-devi-f.png
├── max_devi_f.py
└── param_abacus.json

3 directories, 9 files


- `*.upf` 和 `*.orb` 是 ABACUS 的输入文件
- `param.json` 是运行当前任务的 DP-GEN 设置。
- `machine.json` 是一个任务调度程序，其中设置了计算机环境和资源要求。

### 4.2 DP-GEN 输入文件

DP-GEN 运行过程包含一系列连续迭代，按顺序进行，例如将系统加热到特定温度。每次迭代由三个步骤组成：
* 探索 (Exploration)
* 标记 (Labeling)
* 训练 (Training)。

首先，介绍 DP-GEN 运行过程所需的输入文件。


#### param.json

`param.json` 中的关键字可以分为 4 个部分：

- 系统和数据 (System and Data)：用于指定原子类型、初始数据等。
- 训练 (Training)：主要用于指定训练步骤中的任务;
- 探索 (Exploration)：主要用于在探索步骤中指定任务;
- 标记 (Labeling)：主要用于指定标记步骤中的任务。

这里我们以 LiCl 熔体为例，介绍 `param.json` 中的主要关键词。

In [7]:
! cat LiCl_DP_Tutorial_Example/chapter4/param_abacus.json

{
     "type_map": ["Li","Cl"],
     "mass_map": [6.941,35.453],
     "init_data_prefix": "../chapter3",
     "init_data_sys": ["00.data/training_data"],
     "sys_format": "abacus/stru",
     "sys_configs_prefix": "../chapter2",
     "sys_configs": [["01.md/STRU"]],
     "_comment": " that's all ",

     "numb_models": 4,
     "default_training_param": {
         "model": {
             "type_map": ["Li","Cl"],
             "descriptor": {
                 "type": "se_e2_a",
                 "sel": [128,128],                
                 "rcut_smth": 0.5,
                 "rcut": 7.0,
                 "neuron": [20,40,80],
                 "resnet_dt": false,
                 "axis_neuron": 12,
                 "seed": 1
             },
             "fitting_net": {
                 "neuron": [200,200,200],
                 "resnet_dt": true,
                 "seed": 1
             }
         },
         "learning_rate": {
             "type": "exp",
             "start_lr": 0.001

##### 系统和数据 (System and Data)

系统和数据相关内容：

```
{
     "type_map": ["Li","Cl"],
     "mass_map": [6.941,35.453],
     "init_data_prefix": "../chapter3",
     "init_data_sys": ["00.data/training_data"],
     "sys_format": "abacus/stru",
     "sys_configs_prefix": "../chapter2",
     "sys_configs": [["01.md/STRU"]],
     "_comment": " that's all ",

```

关键词描述：

| 键词                  | 字段类型      | 描述                                                     |
|-----------------------|--------------|----------------------------------------------------------|
| "type_map"            | list         | 元素列表，这里是Li和Cl                                    |
| "mass_map"            | list         | 原子质量列表                                              |
| "init_data_prefix"    | str          | initial data 的前置路径                                   |
| "init_data_sys"       | list         | 初始训练数据文件的路径列表。可以使用绝对路径或相对路径        |
| "sys_format"	        | str          | 指定构型的格式                                            |
| "sys_configs_prefix"  | str          | sys_configs 的前置路径                                    |
| "sys_configs"         | list         | 构型文件的路径列表，此处支持通配符                          |

案例说明：

“type_map”和“mass_map”给出了元素类型和原子质量。在这里，系统包含锂（Li）和氯（Cl）两种，质量分别为6.941和35.453。

“init_data_prefix”和“init_data_sys”关键词共同指定了初始训练数据的位置。

“sys_configs_prefix”和“sys_configs”共同指定了探索的构型的位置。

“sys_format”指定了构型的格式。

在这里，训练数据位于 ./LiCl_DP_Tutorial_Example/chapter3/00.data/training_data目录下。

构型文件位于../ex2/01.md/STRU目录下，采用ABACUS软件的abacus/stru格式

--- 


##### 训练(Training)

与训练相关的内容如下：

```
     "numb_models": 4,
     "default_training_param": {
         "model": {
             "type_map": ["Li","Cl"],
             "descriptor": {
                 "type": "se_e2_a",
                 "sel": [128,128],                
                 "rcut_smth": 0.5,
                 "rcut": 7.0,
                 "neuron": [20,40,80],
                 "resnet_dt": false,
                 "axis_neuron": 12,
                 "seed": 1
             },
             "fitting_net": {
                 "neuron": [200,200,200],
                 "resnet_dt": true,
                 "seed": 1
             }
         },
         "learning_rate": {
             "type": "exp",
             "start_lr": 0.001,
             "decay_steps": 5000
         },
         "loss": {
             "start_pref_e": 0.02,
             "limit_pref_e": 1,
             "start_pref_f": 1000,
             "limit_pref_f": 1,
             "start_pref_v": 0,
             "limit_pref_v": 0
         },
         "training": {
             "numb_steps": 400000,
             "disp_file": "lcurve.out",
             "disp_freq": 1000,
             "numb_test": 1,
             "save_freq": 10000,
             "save_ckpt": "model.ckpt",
             "disp_training": true,
             "time_training": true,
             "profiling": false,
             "profiling_file": "timeline.json",
             "_comment": "that's all"
         }
     },
```

关键词描述：

| 键词                      | 字段类型  | 描述                          |
|---------------------------|----------|-------------------------------|
| "numb_models"             | int      | 在 00.train 中训练的模型数量。  |
| "default_training_param"  | dict     | DeePMD-kit 的训练参数          |

案例说明：

训练相关键指定训练任务的详细信息。`numb_models`指定要训练的模型数量。`default_training_param`指定了 DeePMD-kit 的训练参数。在这里，将训练 4 个 DP 模型。

DP-GEN 的训练部分由 DeePMD-kit 执行，因此此处的关键字与 DeePMD-kit 的关键字相同，此处不再赘述。有关这些关键字的详细说明，请访问[DeePMD-kit 文档](https://deepmd.readthedocs.io/)。

---

##### 探索 (Exploration) 

与探索相关的内容如下：

```
     "model_devi_dt": 0.001,
     "model_devi_skip": 0,
     "model_devi_f_trust_lo": 0.08,
     "model_devi_f_trust_hi": 0.18,
     "model_devi_merge_traj": true,
     "model_devi_clean_traj": false,
     "model_devi_jobs":  [
        {"sys_idx": [0],"temps": [900,1000,1100,1200],"press": [0,10,100,1000,10000], "trj_freq": 10, "nsteps": 100000,"ensemble": "npt", "_idx": "00"},
        {"sys_idx": [0],"temps": [900,1000,1100,1200],"press": [0,10,100,1000,10000], "trj_freq": 10, "nsteps": 100000,"ensemble": "npt", "_idx": "01"},
        {"sys_idx": [0],"temps": [900,1000,1100,1200],"press": [0,10,100,1000,10000], "trj_freq": 10, "nsteps": 100000,"ensemble": "npt", "_idx": "02"}   
     ],
```

关键词描述：

| 键词                      | 字段类型                    | 描述   |
|--------------------------|-------------------------|---------------|
| "model_devi_dt"          | float  | MD 的时间步长                                                                                                                                                                                                                                |
| "model_devi_skip"        | int    | 每个 MD 中为 fp 跳过的结构数                                                                                                                                                                                                |
| "model_devi_f_trust_lo"  | float  | 选择的力下限。如果为 List，则应分别为每个索引设置sys_configs。                                                                                                                                 |
| "model_devi_f_trust_hi"  | int    | 选择的力上限。如果为 List，则应分别为每个索引设置sys_configs。                                                                                                                                  |                                                                                         |
| "model_devi_clean_traj"  | bool or int    | 如果model_devi_clean_traj的类型是布尔类型，则表示是否清理MD中的traj文件。如果是 Int 类型，则将保留 traj 文件夹的最新 n 次迭代，其他迭代将被删除。  |
| "model_devi_clean_traj"  | bool           | 控制在模型偏差（model_devi）阶段是否合并生成的轨迹文件|
| "model_devi_jobs"        | list            | 01.model_devi 中的探索设置。列表中的每个字典对应于一次迭代。model_devi_jobs 的索引与迭代的索引完全一致                                               |
| &nbsp;&nbsp;&nbsp;&nbsp;"sys_idx"   | List of integer         | 选择系统作为MD的初始结构并进行探索。序列与“sys_configs”完全对应。 |
| &nbsp;&nbsp;&nbsp;&nbsp;"temps" | list  | 分子动力学模拟的温度 (K)
| &nbsp;&nbsp;&nbsp;&nbsp;"press" | list  | 分子动力学模拟的压力 (Bar) 
| &nbsp;&nbsp;&nbsp;&nbsp;"trj_freq"   | int          | MD中轨迹的保存频率。                  |
| &nbsp;&nbsp;&nbsp;&nbsp;"nsteps"     | int          | 分子动力学运行步数                                 |
| &nbsp;&nbsp;&nbsp;&nbsp;"ensembles"  | str          | 决定在 MD 中选择的集成算法，选项包括 “npt” ， “nvt”等. |

案例说明

在在“model_devi_jobs”中设置了三次迭代。

对于每次迭代，在不同的温度（900, 1000, 1100和1200 K）和压力条件（0, 1e-3, 1e-2, 1e-1和1 GPa）下，使用 NPT 系综和“sys_configs_prefix”和“sys_configs”指定的构型进行 100000 步模拟，时间步长为0.001 ps。

我们选择保存 MD 轨迹文件，并将保存频率“trj_freq”设置为 10。如果轨迹中构型的“max_devi_f”介于 0.08 和 0.18 之间，DP-GEN 会将该结构视为候选结构。

如果要保存 traj 文件夹的最近 n 次迭代，可以将“model_devi_clean_traj”设置为整数。

---

##### 标记 (Labeling)

与标记相关的内容如下：

```
    "fp_style": "abacus",
    "shuffle_poscar": false,
    "fp_task_max": 200,
    "fp_task_min": 50,
    "fp_pp_path": "./abacus",
    "fp_pp_files": ["Li_ONCV_PBE-1.2.upf","Cl_ONCV_PBE-1.2.upf"],
    "fp_orb_files": ["Li_gga_8au_100Ry_4s1p.orb","Cl_gga_8au_100Ry_2s2p1d.orb"],
    "k_points":[1, 1, 1, 0, 0, 0],
    "user_fp_params":{
    "ntype": 2, 
    "symmetry": 0,
    "vdw_method":"d3_bj",
    "ecutwfc": 100,
    "scf_thr":1e-7,
    "scf_nmax":120, 
    "basis_type":"lcao", 
    "smearing_method": "gauss",
    "smearing_sigma": 0.002,
    "mixing_type": "pulay",
    "mixing_beta": 0.4,
    "cal_force":1,
    "cal_stress":1
```

关键词描述：

| 键词               | 字段类型            | 描述                                                                                                              |
|-------------------|-----------------|---------------------------------------------------------------------|
| "fp_style"        | String          | 第一性原理软件软件。到目前为止，选项包括ABACUS, VASP等。                |
| "shuffle_poscar"  | Boolean         |                                                                     |
| "fp_task_max"     | Integer         | 每次迭代 在 02.fp 中要计算的最大结构。                                 |
| "fp_task_min"     | Integer         | 每次迭代 在 02.fp 中要计算的最小结构。                                 |
| "fp_pp_path"      | String          | 用于 02.fp 的赝势文件路径。                                           |
| "fp_pp_files"     | List of string  | 用于 02.fp 的赝势文件。请注意，元素的顺序应与 type_map 中的顺序相对应。  |
| "fp_orb_files"    | List of string  | 用于 02.fp 的轨道文件。请注意，元素的顺序应与 type_map 中的顺序相对应。  |
| "k_points"        | List of Integer | 用于生成 ABACUS KPT 文件。                                                    |
| "user_fp_params"  | dict            | 用于生成 ABACUS INPUT 文件。如果"user_fp_params"中指定了 kspacing，可以不设置"k_points"。   |

案例说明：

标记相关键词指定标记任务的详细信息。 

在这里，最少 50 个和最多 200 个结构将使用 ABACUS 代码进行标记，在每次迭代中，INPUT 文件依据“user_fp_params”生成，KPT文件依据 “k_points”生成。

请注意，"fp_pp_files" 和 "fp_orb_files" 中元素的顺序应与 `type_map` 中的顺序相对应。

#### machine.json

DP-GEN 运行过程中的每次迭代都由三个步骤组成：探索、标注和训练。因此，machine.json 由三个步骤组成：**train**、**model_devi** 和 **fp**。每个步骤都是字典列表。每个字典都可以被视为一个独立的计算环境。

在本节中，我们将向您展示如何在 Bohrium 上执行`train`, `model_devi`和`fp` 步骤。 对于每个步骤，需要三种类型的关键词：

- 命令 (Command)：提供用于执行每个步骤的命令。
- 机器 (Machine)：指定机器环境（本地工作站、本地或远程集群或云服务器）。
- 资源 (Resources)：指定组、节点、CPU 和 GPU 的数量;启用虚拟环境。

在此示例中，我们在Bohrium上执行`train`步骤。

```json
{
  "api_version": "1.0",
  "deepmd_version": "2.1.5",
  "train" :[
    {
      "command": "dp",
      "machine": {
        "batch_type": "Lebesgue",
        "context_type": "LebesgueContext",
        "local_root" : "./",
        "remote_profile":{
          "email": "xxx", 
          "password": "xxx", 
          "program_id": xxx,
            "keep_backup":true,
            "input_data":{
                "job_type": "container",
                "log_file": "00*/train.log",
                "grouped":true,
                "job_name": "dpgen_train_job",
                "disk_size": 100,
                "scass_type":"c12_m92_1 * NVIDIA V100",
                "checkpoint_files":["00*/checkpoint","00*/model.ckpt*"],
                "checkpoint_time":5,
                "platform": "ali",
                "image_name":"registry.dp.tech/dptech/deepmd-kit:2.1.5-cuda11.6",
                "on_demand":0
            }
        }
      },
      "resources": {
        "local_root":"./",
        "group_size": 1
      }
    }],
  "model_devi":
    [{
      "command": "export LAMMPS_PLUGIN_PATH=/opt/deepmd-kit-2.1.5/lib/deepmd_lmp && lmp -i input.lammps -v restart 0",
      "machine": {
        "batch_type": "Lebesgue",
        "context_type": "LebesgueContext",
        "local_root" : "./",
        "remote_profile":{
          "email": "xxx", 
          "password": "xxx", 
          "program_id": xxx,
            "keep_backup":true,
            "input_data":{
              "job_type": "container",
              "log_file": "task*/model_devi.out",
              "grouped":true,
              "job_name": "dpgen_model_devi_job",
              "disk_size": 200,
              "scass_type":"c12_m92_1 * NVIDIA V100",
              "platform": "ali",
              "image_name":"registry.dp.tech/dptech/deepmd-kit:2.1.5-cuda11.6",
              "checkpoint_files": "sync_files",
              "checkpoint_time":5,
              "on_demand":0
            }
        }
      },
      "resources": {
        "local_root":"./",
        "group_size": 1
      }
    }],
  "fp":
    [{
      "command": "OMP_NUM_THREADS=1 mpirun -np 4 abacus",
      "machine": {
        "batch_type": "Lebesgue",
        "context_type": "LebesgueContext",
        "local_root" : "./",
        "remote_profile":{
          "email": "xxx", 
          "password": "xxx", 
          "program_id": xxx,
            "keep_backup":true,
            "input_data":{
              "log_file": "task*/output",
              "grouped":true,
              "job_name": "dpgen_fp_job",
              "checkpoint_files": "sync_files",
              "checkpoint_time":5,
              "scass_type":"c8_m64_cpu",
              "platform": "ali",
              "image_name":"registry.dp.tech/dptech/abacus:3.1.0",
              "job_type": "container",
              "on_demand":0
            }
        }
      },
      "resources": {
        "group_size": 2,
        "local_root":"./",
        "source_list": []
      }
    }
  ]
}
```

案例说明：

在 command 参数中，`train`，`model_devi`和`fp`步骤使用的程序分别为 DeePMD-kit，LAMMPS 和 ABACUS，其相应的调用命令分别为
```bash
dp
```
```bash
export LAMMPS_PLUGIN_PATH=/opt/deepmd-kit-2.1.5/lib/deepmd_lmp && lmp -i input.lammps -v restart 0 
```
```bash
OMP_NUM_THREADS=1 mpirun -np 4 abacus
```

在 machine 参数中，"scass_type"指定训练使用的机型。

> 对于`train`和`model_devi`步骤，建议使用GPU机型，这里使用的机型为“c12_m92_1 * NVIDIA V100”。
> 
> 对于`fp`步骤，建议使用CPU机型，这里使用的机型为“c8_m64_cpu”。

在 resources 参数中，“group_size”指定了一个 group 中任务的数量。

> 对于`train`和`model_devi`步骤，由于任务数量较少（分别为4和20），我们可以将“group_size”设置为 1, 使所有任务同时执行。
> 对于`fp`步骤，由于任务数量较多（200），我们可以将“group_size”设置为 2, 使100个任务同时执行。

注意：用户需要填入自己的 Bohrium 账户邮箱，密码和项目ID（共三处）。其他参数通常不需修改。

### 4.3 运行DP-GEN

准备好了 `param.json` 和 `machine.json`，我们就可以通过以下方式轻松运行 DP-GEN：

In [27]:
import json
import getpass
from monty.serialization import loadfn, dumpfn

file_path = './LiCl_DP_Tutorial_Example/chapter4/machine.json'
j = loadfn(file_path)

email = input('请输入你的 Bohrium 账户： ')
password = getpass.getpass('请输入你的 Bohrium 密码： ')
program_id = int(input('请输入你的 Bohrium 项目 ID【请输入整数】：'))

dpgen_steps = ['train', 'model_devi', 'fp']

for i in dpgen_steps:
    j[i][0]['machine']['remote_profile']['email'] = email
    j[i][0]['machine']['remote_profile']['password'] = password
    j[i][0]['machine']['remote_profile']['program_id'] = program_id
    
with open(file_path, 'w') as f: 
    json.dump(j, f, indent=4)

请输入你的 Bohrium 账户：  quehaohui@dp.tech
请输入你的 Bohrium 密码：  ········
请输入你的 Bohrium 项目 ID【请输入整数】： 12788


<p style='color:red; font-weight:bold; font-size:2rem'>修改配置文件后，配置文件中携带你的个人账户信息，请注意保密！</p>
通常情况下，该文件不会被分享，你可以运行本文档附录的代码以初始化该配置文件。

In [1]:
# ################# Time Warning: 9 hours ##################
! cd LiCl_DP_Tutorial_Example/chapter4/ && dpgen run param_abacus.json machine.json

/usr/bin/sh: 1: cd: can't cd to LiCl_DP_Tutorial_Example/chapter4/


成功运行 DP-GEN 之后，在 LiCl_DP_Tutorial_Example/chapter4 下，可以发现生成了一个文件夹和两个文件：

- iter.000000 文件夹：包含 DP-GEN 迭代过程中第一次迭代的主要结果；
- dpgen.log 文件：主要记录时间和迭代信息；
- record.dpgen 文件：记录迭代过程的当前阶段。

如果 DP-GEN 的进程由于某种原因停止，DP-GEN 将通过 `record.dpgen`文件自动恢复主进程。我们也可以根据自己的目的手动更改它，例如删除上次迭代并从一个检查点恢复。

`record.dpgen`文件每行包含两个数字：第一个是迭代的索引，第二个是 0 到 9 之间的数字，记录每个迭代中的哪个阶段当前正在运行。

| Index of iterations  | Stage in each iteration     | Process          |
|----------------------|-----------------------------|------------------|
| 0                    | 0                           | make_train       |
| 0                    | 1                           | run_train        |
| 0                    | 2                           | post_train       |
| 0                    | 3                           | make_model_devi  |
| 0                    | 4                           | run_model_devi   |
| 0                    | 5                           | post_model_devi  |
| 0                    | 6                           | make_fp          |
| 0                    | 7                           | run_fp           |
| 0                    | 8                           | post_fp          |

### 4.4 结果分析

第一次迭代完成后，iter.000000 的文件夹结构如下，主要是 3 个文件夹:

In [31]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/ -L 1 

/usr/bin/sh: 1: cd: can't cd to LiCl_DP_Tutorial_Example/chapter4


- 00.train 文件夹：主要是基于现有数据训练的多个 DP 模型（默认是 4 个）；
- 01.model_devi 文件夹：使用 00.train 中得到的 DP 模型进行 MD 模拟，产生新的构型；
- 02.fp 文件夹：对选定的构型进行第一原理计算，并将计算结果转换成训练数据


首先，我们来查看 `iter.000000`/ `00.train`。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/00.train -L 1

iter.000000/00.train
├── data.init -> /data/bohrium-notebook/Proem/CH/LiCl_DP_Tutorial_Example/ex3
└── data.iters

1 directory, 1 file


- 文件夹 00x 包含 DeePMD-kit 的输入和输出文件，其中训练了模型。
- graph.00x.pb ，链接到 00x/frozen.pb，是 DeePMD-kit 生成的模型。这些模型之间的唯一区别是神经网络初始化的随机种子。

让我们随机选择其中之一查看，例如 000。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/00.train/000 -L 1

iter.000000/00.train/000  [error opening dir]

0 directories, 0 files


- `input.json` 是当前任务的 DeePMD-kit 的设置。
- `checkpoint`用于重新开始训练。
- `model.ckpt*` 是与模型相关的文件。
- `frozen_model.pb`是冻结模型。
- `lcurve.out`记录能量和力的训练精度。
- `train.log`包括版本、数据、硬件信息、时间等。


然后，我们来查看`iter.000000/01.model_devi`。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/01.model_devi -L 1

- 文件夹 confs 包含从您在 param.json 的“sys_configs”中设置的 STRU 转换而来的 LAMMPS MD 的初始配置。
- 文件夹 task.000.00000x 包含 LAMMPS 的输入和输出文件。我们可以随机选择其中之一，例如 task.000.000000。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/01.model_devi/task.000.000001

- `conf.lmp`，链接到文件夹 confs 中的“000.0001.lmp”，作为 MD 的初始配置。
- `input.lammps` 是 LAMMPS 的输入文件。
- `model_devi.out` 记录在 MD 训练中相关的标签，能量和力的模型偏差。它是选择结构和进行第一性原理计算的标准。
- `traj` 存储LAMMPS MD轨迹。

通过查看 `model_devi.out` 的前几行, 您会看到:

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && head -n 5 ./iter.000000/01.model_devi/task.000.000001/model_devi.out

想一下，我们将“trj_freq”设置为10，因此每10个步骤保存结构。

是否选择结构取决于其`“max_devi_f”`。如果它介于`“model_devi_f_trust_lo”`（0.08）和`“model_devi_f_trust_hi”`（0.18）之间，DP-GEN 会将该结构视为候选结构。

对于0，10，20 和 30 步保存结构，其`“max_devi_f”`均小于 0.08，所以不会被视为候选结构。

最后，我们来查看 `iter.000000/02.fp` 。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/02.fp

- `candidate.shuffle.000.out` 记录将从最后一步 01.model_devi 中选择哪些结构。 候选的数量总是远远超过您一次期望计算的最大值。在这种情况下，DP-GEN将随机选择最多`fp_task_max`结构并形成文件夹任务。
- `rest_accurate.shuffle.000.out` 记录了我们的模型准确的其他结构（`max_devi_f`小于`model_devi_f_trust_lo`），
- `rest_failed.shuffled.000.out` 记录了我们的模型太不准确的其他结构，这些结构通常是非物理的（大于 `model_devi_f_trust_hi`）。
- `data.000`：经过ABACUS SCF计算后，DP-GEN 将收集这些数据并将其更改为 DeePMD-kit 所需的格式。在下一次迭代的“00.train”中，这些数据将与初始数据一起训练。

通过 `cat candidate.shuffled.000.out | grep task.000.000000`， 你将会看到从task.000.000000中收集的候选构型：

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && cat ./iter.000000/02.fp/candidate.shuffled.000.out | grep task.000.000000

Usage: grep [OPTION]... PATTERNS [FILE]...
Try 'grep --help' for more information.
cat: ./iter.000000/02.fp/candidate.shuffled.000.out: No such file or directory


在第一次迭代之后，我们检查 dpgen.log 的内容。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && cat dpgen.log

可以发现，在 iter.000000 中生成了 200020 个结构，其中候选构型21098个，挑选了 200 个结构进行ABACUS SCF计算，且全部成功收敛。


在所有迭代结束之后，我们来检查 `LiCl_DP_Tutorial_Example/chapter4` 的文件结构：

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree -L 2

以及 `dpgen.log` 的内容：

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && cat cat dpgen.log | grep system

可以发现，仅在三个iteration后accurate_ratio接近100.00 %。在 'iter.000002' 中收集了 9 个候选结构，小于param.json中`fp_task_min`关键词指定的50。因此，模型不会在 iter.000003/00.train 中更新。

为了更直观监测迭代过程，我们提供了提供了一个名为max_devi_f.py的python脚本，用于统计每个iteration中所有构型max_devi_f的分布

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

for i in range(0,3):    # Specify the number of iterations
    max_devi_f_values = []
    for j in range(20):    # 20 LAMMPS tasks in iter*/01.model_devi
        directory = "./LiCl_DP_Tutorial_Example/chapter4/iter.{:06d}/01.model_devi/task.000.{:06d}".format(i, j%20)
        file_path = os.path.join(directory, "model_devi.out")
        data = np.genfromtxt(file_path, skip_header=1, usecols=4)
        max_devi_f_values.append(data)

    # Convert the list to a numpy array
    max_devi_f_values = np.concatenate(max_devi_f_values)

    # Use numpy.histogram() to calculate the frequency of each value
    hist, bin_edges = np.histogram(max_devi_f_values, range=(0, 0.2), bins=100)

    # Normalize the frequency to percentage
    hist = hist / len(max_devi_f_values) * 100

    # Use matplotlib.pyplot.plot() to plot the frequency of each value
    plt.plot(bin_edges[:-1], hist,label = 'iter{:02d}'.format(i))
    plt.legend()
    plt.xlabel("max_devi_f eV/Å")
    plt.ylabel("Distribution %")

    with open(f'./iter{i:02d}_dist-max-devi-f.txt'.format(i), 'w') as f:
        f.write("{:>12} {:>12}\n".format("bin_edges", "hist"))
        for x, y in zip(bin_edges[:-1], hist):
            f.write('{:>12.3f} {:>12.3f}\n'.format(x, y))

plt.savefig('./LiCl_DP_Tutorial_Example/chapter4/max-devi-f.png',dpi=300)
plt.show()

至此，我们应该已经了解了如何使用 DP-GEN 生成和训练深度势模型，以及如何分析输出结果。

## 章节 5：LAMMPS 深度势能分子动力学研究

在训练好 DP 模型后，我们可以将其应用于 LAMMPS 软件中，以便更高效地模拟 LiCl 熔体的性质。这使得我们能够在大尺度和长时间尺度上研究 LiCl 熔体的结构、动力学和热力学性质，从而获得关于材料行为的更深入理解。

### 本章节目标：

学习完本课程后，你将能够：

- 能够利用循环方式进行 LAMMPS 模拟；
- 能够进行 LiCl 熔体的 NPT 和 NVT 计算。

### 5.1 下载教程资源

在本教程中，我们以 LiCl 熔体分子为例，构建训练数据并生成深度势模型。我们已经在 *LiCl_DP_Tutorial_Example/chapter5* 中准备了需要的文件。

首先，使用 `tree` 命令查看 `LiCl_DP_Tutorial_Example/chapter5` 文件夹。


In [ ]:
! tree ./LiCl_DP_Tutorial_Example/chapter5 -L 2  


./LiCl_DP_Tutorial_Example/chapter5
├── 00.npt
│   ├── licl.data
│   └── licl.in
└── 01.nvt
    ├── licl.in
    ├── msd_all_temperatures.png
    └── rdf_all_temperatures.png

2 directories, 5 files


- `licl.in` 和 `licl.data` 是 LAMMPS 的输入文件

本教程采用 DeePMD-kit(2.1.5)软件包中预置的 LAMMSP 程序完成。

### 5.2 NPT-MD模拟



我们在 ex4/iter.000002/01.train/* 文件夹中可以找到最终的4个 DP 模型。通过 `dp compress` 命令，可以将模型压缩，并命名压缩后的模型为 licl_compress_0.pb 到 licl_compress_3.pb。将压缩后的模型分别复制到 00.npt 和 01.nvt 文件夹。接下来，我们进行 LAMMPS NPT-MD 模拟，以预测 LiCl 熔体的密度。LiCl 熔体LAMMPS NPT-MD 的控制文件如下：

```json
variable        a loop 4 pad
variable        b equal $a-1
variable        f equal $b*100
variable        t equal 900+$f

log             log$t.lammps

units           metal
boundary        p p p
atom_style      atomic

read_data       licl.data
replicate       2 2 2 
mass            1 6.94
mass            2 35.45

pair_style      deepmd ./licl_compress_0.pb ./licl_compress_1.pb ./licl_compress_2.pb ./licl_compress_3.pb  out_freq 100 out_file model_devi$t.out
pair_coeff  	* *	


velocity        all create $t 23456789

fix             1 all npt temp $t $t 0.1 iso 0 0 0.5
timestep        0.001

thermo_style    custom step temp pe ke etotal press density lx ly lz vol
thermo          100 

run             1000000
write_data      licl.data$t

clear
next            a
jump            in.licl_npt
```

与 Chapter1 的 licl.in 相比，有几点需要解释：

`variable a loop 4 pad`：创建一个名为a的变量，并通过loop命令在脚本中循环4次。变量 a 将在循环过程中依次取值 1, 2, 3 和 4。

`variable b equal $a-1`：创建一个名为b的变量，其值等于a减1。这意味着在循环中，b的值将从0开始，直到3。

`variable f equal $b*100`：创建一个名为f的变量，其值等于b乘以100。在循环过程中，f的值将是0、100、200和300。

`variable t equal 900+$f`：创建一个名为t的变量，其值等于900加上f。在循环过程中，t的值将是900、1000、1100和1200。

`replicate`  2 2 2：表示沿x、y和z方向将原始系统复制一次。这将使得模拟体系由64原子增大至512原子。

`pair_style  deepmd ./licl_compress_0.pb ./licl_compress_1.pb ./licl_compress_2.pb ./licl_compress_3.pb  out_freq 100 out_file model_devi$t.out`
: 加载了4个神经网络模型,每100个时间步输出一次模型偏差到名为model_devi$t.out的文件，其中 $t 表示温度。

`thermo_style custom step temp pe ke etotal press density lx ly lz vol`: 在thermo_style中增加了density，便于计算模拟体系的密度。

`write_data licl$t.data`: 每次模拟结束时，将模拟体系的信息写入名为 `licl$t.data` 的文件中。

`jump in.licl_npt`: 每次模拟结束后，脚本会清除之前的设置，并跳回到输入文件的开头，准备开始下一次模拟。

我们提供了一个名为 log_lammps.py 的 Python 脚本，可以从 LAMMPS NPT 中获取盒子边长和密度信息。



In [ ]:
import numpy as np

def calculate_mean(file_name, column_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    step_line = next(i for i, line in enumerate(lines) if 'Step' in line)
    loop_line = next(i for i, line in enumerate(lines) if 'Loop' in line)

    column_index = lines[step_line].split().index(column_name)
    data = [float(line.split()[column_index]) for line in lines[step_line+1:loop_line]]

    mean = sum(data[2001:]) / len(data[2001:])      #skip the first 200000 md steps

    return mean

temps=[900,1000,1100,1200]
for temp in temps:
    mean_density = calculate_mean(f'log{temp}.lammps', 'Density')          
    mean_Lx = calculate_mean(f'log{temp}.lammps', 'Lx')
    print(round(mean_density,3), round(mean_Lx,15))


预测密度如下：

表5.1 不同温度下，LiCl熔体的计算密度和相应模拟盒子的边长。

| T(K) |Density(g/cm3)|     Lx           |
|------|--------------|------------------| 
| 900  | 1.602        |22.405256609375005|
| 1000 | 1.562        |22.596664576249935|
| 1100 | 1.523        |22.789268989874945|
| 1200 | 1.485        |22.981117881750063|

### 5.3 NVT-MD 模拟

根据表 5.1 中 Lx 的数值调整 LAMMPS NPT-MD 模拟产生的 licl.data 文件中模拟盒子的边长。接下来，进行 LAMMPS NVT-MD 模拟，以预测 LiCl 熔体的结构信息和离子扩散系数。LiCl 熔体 LAMMPS NVT-MD 的控制文件如下：


```
variable        a loop 4 pad
variable        b equal $a-1
variable        f equal $b*100
variable        t equal 900+$f

log             log$t.lammps

units           metal
boundary        p p p
atom_style      atomic

read_data	licl$t.data
#replicate       2 2 2 
mass 		1 6.94
mass		2 35.45
group		Li  type 1
group		Cl  type 2


pair_style	deepmd ./licl_compress_0.pb ./licl_compress_1.pb ./licl_compress_2.pb ./licl_compress_3.pb  out_freq 100 out_file model_devi$t.out
pair_coeff  	* *	

velocity        all create $t 23456789

fix             1 all nvt temp $t $t 0.5
timestep        0.001

#rdf calculation 
compute 	 rdf all rdf 100 1 1 1 2 2 2
fix 		 2 all ave/time 100 1 100 c_rdf[*] file licl$t.rdf mode vector

#msd calculation
compute          msd1 Li msd
compute          msd2 Cl msd
fix              3 all ave/time 100 1 100 c_msd1[4] c_msd2[4] file licl$t.msd

thermo_style    custom step temp pe ke etotal press density lx ly lz vol
thermo          100 

dump		1 all custom 100 licl$t.dump id type x y z

run             1000000

clear
next            a
jump            licl.in
```


我们可以利用 chapter1 中提供的 Python 脚本，从 licl.rdf 和 licl.msd 中获得径向分布函数和离子自扩散系数。

预测径向分布函数如下：
![rdf_all_temperatures.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/d4a62ccecfca44f7bc04b781453e4308/35302fbd-8e3f-44ff-8be8-18e802e8877c.png)

表5.2 不同温度下，LiCl熔体的中Li-Li，Li-Cl和Cl-Cl离子对径向分布函数第一峰的位置。
| T(K) |Li-Li   |Li-Cl   |Cl-Cl   |
|------|--------|--------|--------| 
| 900  | 3.605  |2.345   |3.675   |
| 1000 | 3.605  |2.345   |3.675   |
| 1100 | 3.605  |2.345   |3.745   |
| 1200 | 3.605  |2.345   |3.745   |

预测均方位移和离子扩散系数如下：
![msd_all_temperatures.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/3267b3d455ba4eb4a9aae6c51bb822f5/c68d0cea-c93a-4c23-b4f2-78e4c39182a6.png)

表5.3 不同温度下，LiCl 熔体的中 $Li^{+}$ 和 $Cl^{-}$ 离子的扩散系数 $\left(m^2/s \times 10^{-9}\right)$。
| T(K) |$D_{Li^{+}}$ | $D_{Cl^{-}}$|
|------|------------------------------------------------|----------------------------------------------| 
| 900  | 8.48          |3.28  |
| 1000 | 10.78        |4.59  |
| 1100 | 12.54        |5.96  |
| 1200 | 18.07        |8.75  |

我们可以将计算结果和文献[[1]](https://doi.org/10.1016/j.jmst.2020.09.040
)进行比较，可以看到计算结构和文献基本吻合。

## 结语

在这项工作中，我们通过新训练的深度势（DPs）研究了典型碱金属氯化物 LiCl 的局部结构和输运特性。我们从从头算分子动力学（AIMD）计算中提取数据集，并用这些数据来训练和验证 DPs。我们在比 AIMD 更广泛的温度范围内进行了大规模、长时间的分子动力学模拟，以确认 DPs 的可靠性和通用性。我们证明了生成的 DPs 模型可以作为模拟碱金属氯化物的有力工具；DPs 还提供了与 AIMD 相当的准确性和与经验势相似的效率。使用 DPs 预测的部分径向分布函数和角分布函数与从 AIMD 得到的结果非常接近。这项工作为 DPs 可用于探索其他系统（包括氯化物混合物或完全不同的盐类）提供了信心。

## 参考文献

[1] Wenshuo Liang, Guimin Lu, Jianguo Yu,Theoretical prediction on the local structure and transport properties of molten alkali chlorides by deep potentials,Journal of Materials Science & Technology 75 (2021) 78–85,[点击访问](https://doi.org/10.1016/j.jmst.2020.09.040
)

---

<a href="https://bohrium.dp.tech/notebook/879b6a3a9d394448913ca1ce7de257ec" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>

## 附录：

In [17]:
# 运行以下代码以初始化 machine.json 配置文件

import json
import getpass
from monty.serialization import loadfn, dumpfn

file_path = './LiCl_DP_Tutorial_Example/chapter4/machine.json'
j = loadfn(file_path)

dpgen_steps = ['train', 'model_devi', 'fp']

for i in dpgen_steps:
    j[i][0]['machine']['remote_profile']['email'] = 'email'
    j[i][0]['machine']['remote_profile']['password'] = 'password'
    j[i][0]['machine']['remote_profile']['program_id'] = 12345
    
with open(file_path, 'w') as f: 
    json.dump(j, f, indent=4)
    print('初始化配置文件完成！')

初始化配置文件完成！


In [ ]:
# ########### 注意，以下操作会删除本 Notebook 所有输出。
# ! if ! pip show nbstripout > /dev/null; then pip install nbstripout; fi;
# ! nbstripout LiCl_DP_Tutoria.ipynb

---